In [ ]:
import os
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_file_names = pd.read_csv("/content/drive/My Drive/Colab Notebooks/repozitar-downloads/repozitory-export.csv", sep=';')
display(df_file_names.head(3))

,file_id,file_name,file_type,cc_license,title,journal_title,journal_issn,journal_year,doi
0,19,Corruption-judicial-accountability-and-inequal...,published,none,"Corruption, judicial accountability and inequa...",Journal of Economic Behavior & Organization,0167-2681,2020,10.1016/j.jebo.2019.12.010
1,208,3-24-10-Final.pdf,accepted,none,Smart City Model Based on Systems Theory,International Journal of Information Management,0268-4012,2020,10.1016/j.ijinfomgt.2020.102092
2,213,The-evaluation-of-project-management-practices...,published,none,The Evaluation of Project Management Practices...,Economic Research-Ekonomska Istraživanja,1331-677X,2019,10.1080/1331677X.2018.1553679


In [ ]:
def rename_file(file_path, old_name, new_name, errors_list):
    if old_name == new_name:
        return

    old_path = os.path.join(file_path, old_name)
    new_path = os.path.join(file_path, new_name)

    # Kontrola, zda původní soubor existuje
    if not os.path.exists(old_path):
        errors_list.append((old_name, "Soubor nenalezen"))
        return

    # Kontrola, zda nový soubor už neexistuje
    if os.path.exists(new_path):
        errors_list.append((old_name, f"Nový název {new_name} už existuje"))
        return

    try:
        os.rename(old_path, new_path)
    except Exception as e:
        errors_list.append((old_name, str(e)))



In [ ]:
#nespouštět!!!
cesta = '/content/drive/My Drive/Colab Notebooks/repozitar-downloads/'
errors = []

# Iterace přes CSV
for row in df_file_names.itertuples(index=False):
    old = str(getattr(row, "file_id")) + ".data"
    new_one = getattr(row, "file_name")
    rename_file(cesta, old, new_one, errors)

# Výpis výsledků
if errors:
    print("⚠️ Došlo k chybám při přejmenování:")
    for e in errors:
        print(e)
else:
    print("✅ Všechny soubory byly úspěšně přejmenovány.")

⚠️ Došlo k chybám při přejmenování:
('19.data', 'Soubor nenalezen')
('208.data', 'Soubor nenalezen')
('601.data', 'Soubor nenalezen')
('786.data', 'Nový název materials-978466.pdf už existuje')
('912.data', 'Nový název sustainability-12-09502.pdf už existuje')
('1077.data', 'Nový název pep-001086-fin-0002.pdf už existuje')
('1402.data', 'Soubor nenalezen')
('1480.data', 'Nový název buildings-12-01594.pdf už existuje')
('1731.data', 'Nový název 32019723q4-vozar-analyses.pdf už existuje')
('1826.data', 'Nový název geo-2021126040419.pdf už existuje')
('1834.data', 'Nový název acceptedauthorpublishingagreement.pdf už existuje')
('1902.data', 'Nový název 0cef2476-5536-40f2-b873-87ee4ba1a6d0.pdf už existuje')
('2224.data', 'Nový název am-2-2024-4-8-.pdf už existuje')


In [ ]:
df_file_names.shape

(387, 9)

In [ ]:
#mazání nekonvertovaných souborů - nespouštět

# Cesta ke složce
cesta = '/content/drive/My Drive/Colab Notebooks/repozitar-downloads/'

# Původní DataFrame
df = df_file_names.copy()

# Seznam indexů, které se mají odstranit z df
rows_to_drop = []

# Iterace přes DataFrame
for idx, row in df.iterrows():
    old_name = f"{row['file_id']}.data"
    file_path = os.path.join(cesta, old_name)

    if os.path.exists(file_path):
        try:
            os.remove(file_path)  # smaže soubor
            rows_to_drop.append(idx)  # označí řádek k odstranění
            print(f"🗑️ Smazán soubor: {old_name}")
        except Exception as e:
            print(f"⚠️ Chyba při mazání {old_name}: {e}")

# Odstranění odpovídajících řádků z DataFrame
df = df.drop(index=rows_to_drop).reset_index(drop=True)

# Výsledek
print(f"\n✅ Hotovo! Smazáno {len(rows_to_drop)} souborů a odpovídajících záznamů z datasetu.")

# Pokud chceš přepsat CSV soubor novou verzí:
# df.to_csv("/content/drive/My Drive/Colab Notebooks/repozitar-downloads/repozitory-export-clean.csv", sep=';', index=False)


🗑️ Smazán soubor: 786.data
🗑️ Smazán soubor: 912.data
🗑️ Smazán soubor: 1077.data
🗑️ Smazán soubor: 1480.data
🗑️ Smazán soubor: 1731.data
🗑️ Smazán soubor: 1826.data
🗑️ Smazán soubor: 1834.data
🗑️ Smazán soubor: 1902.data
🗑️ Smazán soubor: 2224.data

✅ Hotovo! Smazáno 9 souborů a odpovídajících záznamů z datasetu.


In [ ]:
import requests, re
import time

API_KEY = "1"
HEADERS = {
    "X-ApiKey": API_KEY,
    "Accept": "application/json",
    "User-Agent": "python-clarivate-test/1.0"
}
URL = "https://api.clarivate.com/apis/wos-journals/v1"

def norm_issn(x):
    m = re.search(r"(\d{4})-?(\d{3}[\dXx])", str(x)) if x else None
    return f"{m.group(1)}-{m.group(2).upper()}" if m else None

def safe_json(resp):
    try:
        return resp.json()
    except Exception:
        # diagnostika: status + typ obsahu + začátek těla
        ct = resp.headers.get("Content-Type", "")
        print(f" Nejde parsovat JSON. Status: {resp.status_code}, Content-Type: {ct}")
        print("↳ Náhled odpovědi:", (resp.text or "")[:400])
        return None

def jcr_by_issn(issn, year=None):
    issn = norm_issn(issn)
    if not issn:
        print(" ISSN není platné.")
        return None

    params = {"issn": issn}
    if year:
        params["year"] = year

    r = requests.get(URL, headers=HEADERS, params=params, timeout=30)

    if r.status_code in (401, 403):
        print(f" Přístup zamítnut ({r.status_code}). Pravděpodobně chybí subscription k JCR API.")
        print("↳", (r.text or "")[:400])
        return None
    if r.status_code >= 400:
        print(f" HTTP chyba {r.status_code}")
        print("↳", (r.text or "")[:400])
        return None

    payload = safe_json(r)
    if not payload or not payload.get("data"):
        print(" Odpověď neobsahuje 'data' (viz výše diagnostiku).")
        return None

    d = payload["data"][0]
    m = d.get("metrics") or {}

    print(f" Časopis: {d.get('title')} (ISSN {issn})")
    print(f" Rok: {m.get('year')}")
    print(f"JIF: {m.get('jif')}")
    print(f" AIS: {m.get('articleInfluence')}")
    return m

# --- test ---
first_row = df_file_names.iloc[45]
issn = first_row["journal_issn"]
print(f" Test: ISSN = {issn}")
jcr_by_issn(issn)
time.sleep(0.3)#omezení vzhledem k počtu možných requestů za vteřinu


 Test: ISSN = 1232-8855
 Přístup zamítnut (403). Pravděpodobně chybí subscription k JCR API.
↳ {
  "message":"You cannot consume this service"
}


In [ ]:
df_first = pd.read_csv("/content/drive/My Drive/Colab Notebooks/datasets/M1_VO_posudky_H20-H24.csv", sep=';')
df_second = pd.read_csv("/content/drive/My Drive/Colab Notebooks/datasets/publications_final.csv", sep=';')

In [ ]:
df_second.head()

,title,Vědní obor,year,Počet tvůrců celkem,Seznam tvůrců,Název organizace,abstract,keywords,category,journal,AIS,IF,FORD_WOS,final_grade
0,A transparent insulation facade enhanced with ...,20101,2019,2,"Čekon, Miroslav:Čurpek, Jakub",Vysoké učení technické v Brně,Solar thermal façade concepts for building int...,"Solar wall, Solar façade, Transparent insulati...",J,ENERGY AND BUILDINGS,0.811,4.867,20100.0,2
1,A LHCB9-dependent photosystem I megacomplex in...,10610,2018,12,"Pinnola, Alberta:Alboresi, Alessandro:Nosek, L...",Univerzita Palackého v Olomouci,Photosystem I of the moss Physcomitrella paten...,electron microscopy:Physcomitrella patens:mega...,J,Nature Plants,5.535,13.297,10600.0,2
2,A link between knowledge sharing and managers’...,50204,2018,3,"Matošková, Jana:Macurová, Lucie:Tomancová, Lucie",Univerzita Tomáše Bati ve Zlíně,Purpose – The purpose of this paper is to exam...,"Knowledge sharing, Supervisor, Knowledge-orien...",J,Leadership and Organization Development Journal,0.301,1.462,50200.0,3
3,A non-radial directional distance method on cl...,10102,2018,3,"Toloo, Mehdi:Allahyar, Maryam:Hančlová, Jana",Vysoká škola báňská - Technická univerzita Ost...,The original Data Envelopment Analysis (DEA) m...,industry:banking:Application:DEA:outputs:and:i...,J,Expert Systems with Applications,0.790,4.292,10200.0,2
4,A Novel Method for Detection of Covered Conduc...,20201,2018,6,"Krátký, Michal:Mišák, Stanislav:Gajdoš, Petr:L...",Vysoká škola báňská - Technická univerzita Ost...,Although there are many reasons why medium vol...,PD-pattern:partial discharges (PD):medium volt...,J,IEEE Transactions on Industrial Electronics,2.035,7.503,20200.0,4


In [ ]:
df_first.head()

,Segment,Poskytovatel,IČO,Název,Organizační jednotka,Evidenční číslo,Druh,Kritérium,Autoři,Název výsledku,Vědní oblast,Obor (Ford),D-Ford,Období,Vypracoval,Text,Dílčí známka,Finální známka,33 364,Unnamed: 19
0,Rezorty,Ministerstvo zemědělství,26784246,Agritec Plant Research s.r.o.,Agritec Plant Research s.r.o.,192139910,J,Společenská relevance,"Poslušná, Jana;Plachká, Eva;Horáček, Jiří;Mach...","The Harmfulness of Phoma Stem Canker, Scleroti...",4. Agricultural and veterinary sciences,"4.1 Agriculture, Forestry, and Fisheries",-,H20,Hodnotitel,Ozimá řepka je v současné době u nás druhou ne...,3,3,NaN,NaN
1,Rezorty,Ministerstvo zemědělství,26784246,Agritec Plant Research s.r.o.,Agritec Plant Research s.r.o.,192139910,J,Společenská relevance,"Poslušná, Jana;Plachká, Eva;Horáček, Jiří;Mach...","The Harmfulness of Phoma Stem Canker, Scleroti...",4. Agricultural and veterinary sciences,"4.1 Agriculture, Forestry, and Fisheries",-,H20,Hodnotitel,Výsledkem je publikace článku (The Harmfulness...,3,3,NaN,NaN
2,Rezorty,Ministerstvo zemědělství,26784246,Agritec Plant Research s.r.o.,Agritec Plant Research s.r.o.,192139911,N,Společenská relevance,"Huňady, Igor;Hanuš, Oto;Seidenglanz, Marek;Poz...",Metodika pěstování luskovino-obilních směsek z...,4. Agricultural and veterinary sciences,"4.1 Agriculture, Forestry, and Fisheries",-,H20,Hodnotitel,Cílem předloženého výstupu je podpořit rozvoj ...,2,2,NaN,NaN
3,Rezorty,Ministerstvo zemědělství,26784246,Agritec Plant Research s.r.o.,Agritec Plant Research s.r.o.,192139911,N,Společenská relevance,"Huňady, Igor;Hanuš, Oto;Seidenglanz, Marek;Poz...",Metodika pěstování luskovino-obilních směsek z...,4. Agricultural and veterinary sciences,"4.1 Agriculture, Forestry, and Fisheries",-,H20,Hodnotitel,Velmi kvalitně a přehledně zpracovaná metodika...,2,2,NaN,NaN
4,Rezorty,Ministerstvo zemědělství,26784246,Agritec Plant Research s.r.o.,Agritec Plant Research s.r.o.,192184580,N,Společenská relevance,"Smýkalová, Iva;Doležal, Karel",Biotechnologická produkce konopí setého (Canna...,4. Agricultural and veterinary sciences,4.4 Agricultural biotechnology,-,H20,Hodnotitel,Předložený výsledek je reprezentován nově vyvi...,4,4,NaN,NaN


In [ ]:
df_first['Text'] = df_first['Text'].astype(str).str.replace(r'[\r\n]+', ' ', regex=True)


In [ ]:
df_first_filtered = df_first[['Název výsledku', 'Období', 'Vypracoval', 'Text','Dílčí známka', 'Finální známka',]]
df_first_filtered.head()

,Název výsledku,Období,Vypracoval,Text,Dílčí známka,Finální známka
0,"The Harmfulness of Phoma Stem Canker, Scleroti...",H20,Hodnotitel,Ozimá řepka je v současné době u nás druhou ne...,3,3
1,"The Harmfulness of Phoma Stem Canker, Scleroti...",H20,Hodnotitel,Výsledkem je publikace článku (The Harmfulness...,3,3
2,Metodika pěstování luskovino-obilních směsek z...,H20,Hodnotitel,Cílem předloženého výstupu je podpořit rozvoj ...,2,2
3,Metodika pěstování luskovino-obilních směsek z...,H20,Hodnotitel,Velmi kvalitně a přehledně zpracovaná metodika...,2,2
4,Biotechnologická produkce konopí setého (Canna...,H20,Hodnotitel,Předložený výsledek je reprezentován nově vyvi...,4,4


In [ ]:
df_merged = pd.merge(df_second, df_first_filtered, left_on='title', right_on='Název výsledku')
display(df_merged.head(3))

,title,Vědní obor,year,Počet tvůrců celkem,Seznam tvůrců,Název organizace,abstract,keywords,category,journal,AIS,IF,FORD_WOS,final_grade,Název výsledku,Období,Vypracoval,Text,Dílčí známka,Finální známka
0,Arcades: Products of stress-controlled and dis...,10505,2018,6,"Filippi, Michal:Bruthans, J.:Řihošek, J.:Slaví...","Geologický ústav AV ČR, v. v. i.",Weathering patterns on sandstone cliff faces a...,arcades:cavernous weathering:discontinuity:gra...,I,Earth-Science Reviews,3.613,9.53,10500.0,2,Arcades: Products of stress-controlled and dis...,H22,Panelista Garant,Výsledek „Arcades: Products of stress-controll...,2,2
1,Arginine-rich cell-penetrating peptides induce...,10403,2018,15,"Allolio, Christoph:Magarkar, Aniket:Jurkiewicz...",Ústav fyzikální chemie Jaroslava Heyrovského A...,Arginine-rich cell-penetrating peptides do not...,cell-penetrating peptide:membrane fusion:fluor...,I,Proceedings of the National Academy of Science...,4.493,9.58,10700.0,1,Arginine-rich cell-penetrating peptides induce...,H21,Hodnotitel,The authors focused on explanation of how the ...,2,2
2,Arginine-rich cell-penetrating peptides induce...,10403,2018,15,"Allolio, Christoph:Magarkar, Aniket:Jurkiewicz...",Ústav fyzikální chemie Jaroslava Heyrovského A...,Arginine-rich cell-penetrating peptides do not...,cell-penetrating peptide:membrane fusion:fluor...,I,Proceedings of the National Academy of Science...,4.493,9.58,10700.0,1,Arginine-rich cell-penetrating peptides induce...,H21,Hodnotitel,Bezpochyby výsledek na špičkové úrovni. Autoři...,1,2


In [ ]:
import csv, sys

# Cesty k souborům na Google Drive
inp = "/content/drive/My Drive/Colab Notebooks/df_merged.csv"
out = "/content/drive/My Drive/Colab Notebooks/df_merged_oneline.csv"

with open(inp, "r", encoding="utf-8", newline="") as f_in, \
     open(out, "w", encoding="utf-8", newline="") as f_out:
    reader = csv.reader(f_in)  # automaticky respektuje uvozovky a oddělovač ,
    writer = csv.writer(f_out, lineterminator="\n")
    for row in reader:
        # nahradíme v každé buňce vnitřní \r/\n za sekvenci \n (nebo za mezeru)
        row = [cell.replace("\r\n", "\\n").replace("\n", "\\n").replace("\r", "\\n")
               for cell in row]
        writer.writerow(row)

print("Hotovo ->", out)


Hotovo -> /content/drive/My Drive/Colab Notebooks/df_merged_oneline.csv


In [ ]:
df_merged.shape

(8655, 20)

In [ ]:
df_merged_filtered.shape

(8655, 13)

In [ ]:
output_path = "/content/drive/My Drive/Colab Notebooks/df_merged_filtered.csv"

In [ ]:
df_loaded = pd.read_csv(output_path)
display(df_loaded.head(100))

,title,Počet tvůrců celkem,Seznam tvůrců,Název organizace,abstract,keywords,category,journal,AIS,IF,Text,Období,final_grade
0,Arcades: Products of stress-controlled and dis...,6,"Filippi, Michal:Bruthans, J.:Řihošek, J.:Slaví...","Geologický ústav AV ČR, v. v. i.",Weathering patterns on sandstone cliff faces a...,arcades:cavernous weathering:discontinuity:gra...,I,Earth-Science Reviews,3.613,9.530,Výsledek „Arcades: Products of stress-controll...,H22,2
1,Arginine-rich cell-penetrating peptides induce...,15,"Allolio, Christoph:Magarkar, Aniket:Jurkiewicz...",Ústav fyzikální chemie Jaroslava Heyrovského A...,Arginine-rich cell-penetrating peptides do not...,cell-penetrating peptide:membrane fusion:fluor...,I,Proceedings of the National Academy of Science...,4.493,9.580,The authors focused on explanation of how the ...,H21,1
2,Arginine-rich cell-penetrating peptides induce...,15,"Allolio, Christoph:Magarkar, Aniket:Jurkiewicz...",Ústav fyzikální chemie Jaroslava Heyrovského A...,Arginine-rich cell-penetrating peptides do not...,cell-penetrating peptide:membrane fusion:fluor...,I,Proceedings of the National Academy of Science...,4.493,9.580,Bezpochyby výsledek na špičkové úrovni. Autoři...,H21,1
3,Arginine-rich cell-penetrating peptides induce...,15,"Allolio, Christoph:Magarkar, Aniket:Jurkiewicz...",Ústav fyzikální chemie Jaroslava Heyrovského A...,Arginine-rich cell-penetrating peptides do not...,cell-penetrating peptide:membrane fusion:fluor...,I,Proceedings of the National Academy of Science...,4.493,9.580,Multioborový výsledek Dříve hodnocen jako špič...,H21,1
4,Beyond the classical thermodynamic contributio...,4,"Bím, Daniel:Maldonado-Domínguez, Mauricio:Rulí...",Ústav fyzikální chemie Jaroslava Heyrovského A...,Hydrogen atom abstraction (HAA) reactions are ...,acidity constant:asynchronicity factor:hydroge...,I,Proceedings of the National Academy of Science...,4.493,9.580,"Uvedený výsledek je originální, z hlediska výz...",H20,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,A Rotational BODIPY Nucleotide: An Environment...,5,"Dziuba, Dmytro:Jurkiewicz, Piotr:Cebecauer, Ma...","Ústav organické chemie a biochemie AV ČR, v. v...",Fluorescent probes for detecting the physical ...,DNA:fluorescence spectroscopy:fluorescent prob...,I,Angewandte Chemie - International Edition,3.433,11.994,In this “Hot paper” published in well-known sc...,H20,1
96,Coexistence of gain-of-function JAK2 germ line...,8,"Láníková, Lucie:Babošová, Oľga:Swierczek, S.:W...","Ústav molekulární genetiky AV ČR, v. v. i.",We described the association of gain-of-functi...,JAK2 mutations:polycythemia vera,I,Blood,4.022,13.164,Tato studie prezentuje pravděpodobně první pop...,H20,2
97,Coexistence of gain-of-function JAK2 germ line...,8,"Láníková, Lucie:Babošová, Oľga:Swierczek, S.:W...","Ústav molekulární genetiky AV ČR, v. v. i.",We described the association of gain-of-functi...,JAK2 mutations:polycythemia vera,I,Blood,4.022,13.164,Hodnocená práce se zabývá funkční analýzou dvo...,H20,2
98,"Development of 3,5-Dinitrobenzylsulfanyl-1,3,4...",19,"Karabanovich, Galina:Zemanová, Júlia:Smutný, T...",Univerzita Karlova,"Herein, we report the discovery and structure ...",oxadiazole:inhibitors:prediction:cytotoxicity:...,J,Journal of Medicinal Chemistry,1.516,6.259,Publikační výstup popisuje syntézu a charakter...,H20,1


In [ ]:
from openai import OpenAI
import json
client = OpenAI(api_key="2")
models = client.models.list()

In [ ]:
import re
import pandas as pd

def shrnuti_hodnoceni(hodnoceni_list, paper_row):
    # hodnoceni_list je seznam 10 textových hodnocení
    # paper_row je pandas Series s informacemi o náhodně vybraném článku (např. title, abstract, IF, category…)
    text = "\n".join(f"- {h}" for h in hodnoceni_list)


    article_desc_parts = []
    for col in ["title", "abstract", "journal", "IF", "category", "keywords"]:
        if col in paper_row and isinstance(paper_row[col], (str, int, float)):
            article_desc_parts.append(f"{col}: {paper_row[col]}")
    article_desc = "\n".join(article_desc_parts) if article_desc_parts else "bez detailů"

    response = client.chat.completions.create(
        model="gpt-5-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    "Jsi asistent, který STRUČNĚ shrnuje textová hodnocení a na jejich základě odhaduje známku dle metodiky M17+ (1–7).\n\n"
                    "1) Nejprve napiš JEDEN krátký odstavec shrnutí ve stylu:\n"
                    "\"Hodnocení (10) článků vesměs hodnotilo především <hlavní oblasti>; častěji chválilo <...>, kritizovalo zejména <...>. "
                    "V menší míře se řešilo <...>.\"\n"
                    "- Vytěž klíčové tematické okruhy z poskytnutých 10 hodnocení (např. originalita, metodologie, statistika, přínos/impact, "
                    "gramatika a stylistika, struktura, práce s literaturou/citace, etika, kvalita dat, design studie, jasnost abstraktu).\n"
                    "- Buď konkrétní (co se typicky chválilo/kritizovalo), ale úsporný (≈60–120 slov). Bez odkazů a bez odrážek.\n\n"
                    "2) Poté PREDIKUJ známku M17+ (1–7) pro zadaný článek takto:\n"
                    "- Primárně zvaž soulad zjištěných priorit/požadavků ze shrnutí s obsahem a kvalitou METADAT článku "
                    "(zejm. abstrakt → gramatika/srozumitelnost, jasnost cílů, metodologie, přínos; dále relevanci ke kategorii a klíčovým slovům).\n"
                    "- Journal prestiž (IF/AIS, pokud jsou uvedeny) ber jako slabší až střední doplňkový signál, ne jako hlavní.\n"
                    "- Nepoužívej žádné externí znalosti mimo poskytnutý text.\n"
                    "- Pokud shrnutí zdůrazňuje např. gramatiku/čitelnost a abstrakt je zjevně problematický, přiměřeně penalizuj; "
                    "pokud naopak splňuje, přičti.\n"
                    "- Přibližné mapování M17+: 1–2 excelentní, 3–4 dobré/průměrné, 5–6 slabé, 7 nevyhovující.\n\n"
                    "Formát odpovědi MUSÍ být přesně:\n"
                    "Shrnutí: <stručný odstavec>\n"
                    "Predikovaná_znamka: <číslo 1-7>\n\n"
                    "Nevysvětluj postup, neuváděj odůvodnění mimo shrnutí, nic dalšího nepřidávej."
                ),
            },
            {
                "role": "user",
                "content": (
                    f"Zde jsou vzorová hodnocení (10 položek):\n{text}\n\n"
                    f"Predikuj známku pro tento článek (použij výhradně kontext výše a níže uvedená metadata):\n{article_desc}\n\n"
                    "Připomeň si: shrnutí začni větou \"Hodnocení (10) článků vesměs hodnotilo především ...\", "
                    "poté jasně uveď co se chválilo a co kritizovalo; známku určuj primárně podle souladu shrnutých priorit "
                    "s obsahem a kvalitou abstraktu a dalšími metadaty (title, abstract, journal, IF/AIS, category, keywords). "
                    "Dodrž formát:\n"
                    "Shrnutí: ...\n"
                    "Predikovaná_znamka: X"
                ),
            },
        ],
    )

    raw = response.choices[0].message.content.strip()


    m = re.search(r"Predikovan[aá]_?znamka:\s*([1-7])", raw, flags=re.IGNORECASE)
    predicted_grade = int(m.group(1)) if m else None
    return raw, predicted_grade


# --- hlavní běh ---
vysledky = []

for i in range(10):
    # 10 náhodných hodnocení + 1 náhodný článek
    random_reviews = df_loaded.sample(10)
    random_paper = df_loaded.sample(1).iloc[0]
    reviews_text = random_reviews['Text'].astype(str).tolist()

    try:
        summary_text, pred_grade = shrnuti_hodnoceni(reviews_text, random_paper)
    except Exception as e:

        summary_text, pred_grade = f"Chyba při volání modelu: {e}", None

    # skutečná známka z dat (pokud je sloupec k dispozici)
    true_grade = random_paper['final_grade'] if 'final_grade' in random_paper else None


    vysledky.append({
        "run": i + 1,
        "paper_title": random_paper.get("title", None),
        "skutecna_znamka": true_grade,
        "predikovane_cislo": pred_grade,
        "predikovany_text": summary_text
    })

# převod do DataFrame a uložení na disk (CSV)
vysledky_df = pd.DataFrame(vysledky)
vysledky_df.to_csv("m17_predikce_10x.csv", index=False, encoding="utf-8")

# --- výstup do konzole ---
print("Souhrn 10 běhů (skutečná vs. predikovaná):")
for r in vysledky:
    print(f"{r['run']:>2}. Skutečná: {r['skutecna_znamka']} | Predikovaná: {r['predikovane_cislo']} | Titul: {r['paper_title']}")

print("\n---\nDetailní shrnutí každého běhu:")
for r in vysledky:
    print(f"\n=== Běh {r['run']} ===")
    print(r["predikovany_text"])

print("\nUloženo do souboru: m17_predikce_10x.csv")


Souhrn 10 běhů (skutečná vs. predikovaná):
 1. Skutečná: 2 | Predikovaná: 1 | Titul: Hydrogen peroxide coordination to cobalt(II) facilitated by second-sphere hydrogen bonding
 2. Skutečná: 4 | Predikovaná: 3 | Titul: Intercellular mitochondrial transfer in the tumor microenvironment
 3. Skutečná: 2 | Predikovaná: 1 | Titul: Conformation in ultrathin polymer brush coatings resolved by infrared nanoscopy
 4. Skutečná: 4 | Predikovaná: 2 | Titul: Resilience Assessment in Electricity Critical Infrastructure from the Point of View of Converged Security
 5. Skutečná: 2 | Predikovaná: 1 | Titul: Bispecific IgG neutralizes SARS-CoV-2 variants and prevents escape in mice
 6. Skutečná: 1 | Predikovaná: 2 | Titul: Helminth Therapy From the Parasite Perspective
 7. Skutečná: 3 | Predikovaná: 1 | Titul: Mycobiome diversity: high-throughput sequencing and identification of fungi
 8. Skutečná: 3 | Predikovaná: 2 | Titul: Journalistic discourse of freedom: A study of journalists' understanding of fre

In [ ]:
#korelace IF/AIS s hodnocením
import pandas as pd

#  potřebné sloupce
df_corr = df_merged[['final_grade', 'IF', 'AIS']].copy()

# převod na čísla
df_corr['final_grade'] = pd.to_numeric(df_corr['final_grade'], errors='coerce')
df_corr['IF'] = pd.to_numeric(df_corr['IF'], errors='coerce')
df_corr['AIS'] = pd.to_numeric(df_corr['AIS'], errors='coerce')

# odstranění řádků s NaN
df_corr = df_corr.dropna()

# korelace
cor_if = df_corr['final_grade'].corr(df_corr['IF'])
cor_ais = df_corr['final_grade'].corr(df_corr['AIS'])

print(f'Korelace final_grade s IF: {cor_if:.3f}')
print(f'Korelace final_grade s AIS: {cor_ais:.3f}')



Korelace final_grade s IF: -0.264
Korelace final_grade s AIS: -0.313


In [ ]:
df_loaded.head()

,title,Počet tvůrců celkem,Seznam tvůrců,Název organizace,abstract,keywords,category,journal,AIS,IF,Text,Období,final_grade
0,Arcades: Products of stress-controlled and dis...,6,"Filippi, Michal:Bruthans, J.:Řihošek, J.:Slaví...","Geologický ústav AV ČR, v. v. i.",Weathering patterns on sandstone cliff faces a...,arcades:cavernous weathering:discontinuity:gra...,I,Earth-Science Reviews,3.613,9.53,Výsledek „Arcades: Products of stress-controll...,H22,2
1,Arginine-rich cell-penetrating peptides induce...,15,"Allolio, Christoph:Magarkar, Aniket:Jurkiewicz...",Ústav fyzikální chemie Jaroslava Heyrovského A...,Arginine-rich cell-penetrating peptides do not...,cell-penetrating peptide:membrane fusion:fluor...,I,Proceedings of the National Academy of Science...,4.493,9.58,The authors focused on explanation of how the ...,H21,1
2,Arginine-rich cell-penetrating peptides induce...,15,"Allolio, Christoph:Magarkar, Aniket:Jurkiewicz...",Ústav fyzikální chemie Jaroslava Heyrovského A...,Arginine-rich cell-penetrating peptides do not...,cell-penetrating peptide:membrane fusion:fluor...,I,Proceedings of the National Academy of Science...,4.493,9.58,Bezpochyby výsledek na špičkové úrovni. Autoři...,H21,1
3,Arginine-rich cell-penetrating peptides induce...,15,"Allolio, Christoph:Magarkar, Aniket:Jurkiewicz...",Ústav fyzikální chemie Jaroslava Heyrovského A...,Arginine-rich cell-penetrating peptides do not...,cell-penetrating peptide:membrane fusion:fluor...,I,Proceedings of the National Academy of Science...,4.493,9.58,Multioborový výsledek Dříve hodnocen jako špič...,H21,1
4,Beyond the classical thermodynamic contributio...,4,"Bím, Daniel:Maldonado-Domínguez, Mauricio:Rulí...",Ústav fyzikální chemie Jaroslava Heyrovského A...,Hydrogen atom abstraction (HAA) reactions are ...,acidity constant:asynchronicity factor:hydroge...,I,Proceedings of the National Academy of Science...,4.493,9.58,"Uvedený výsledek je originální, z hlediska výz...",H20,2


In [ ]:
df_loaded["journal"].value_counts()

,count
journal,
Nature Communications,182
Scientific Reports,119
Physical Review Letters,101
Nucleic Acids Research,92
Science of the Total Environment,92
...,...
Industrial & Engineering Chemistry Research,1
ENVIRONMENTAL SCIENCE & TECHNOLOGY,1
SIAM Journal on Discrete Mathematics,1


In [ ]:
# články z určitého journalu
df_test = df_loaded[df_loaded["journal"] == "Scientific Reports"].copy()

# důležité sloupce
df_test = df_test[['final_grade', 'IF', 'AIS']].copy()

# převod na číselné hodnoty
df_test['final_grade'] = pd.to_numeric(df_test['final_grade'], errors='coerce')
df_test['IF'] = pd.to_numeric(df_test['IF'], errors='coerce')
df_test['AIS'] = pd.to_numeric(df_test['AIS'], errors='coerce')

# odstranění chybějících hodnot
df_test = df_test.dropna()

# výpočet korelací
cor_if = df_test['final_grade'].corr(df_test['IF'])
cor_ais = df_test['final_grade'].corr(df_test['AIS'])

print(f'Korelace final_grade s IF: {cor_if:.3f}')
print(f'Korelace final_grade s AIS: {cor_ais:.3f}')


Korelace final_grade s IF: 0.392
Korelace final_grade s AIS: -0.373


In [ ]:
import time
import math
import requests
import pandas as pd

df_sample = df_file_names.dropna(subset=["doi"]).sample(10, random_state=42)

API_KEY = "b181db93f023450eb81cf28da22ede609da1ef88"
BASE_URL = "https://api.clarivate.com/apis/wos-starter/v1/documents"
HEADERS = {"X-ApiKey": API_KEY, "Accept": "application/json"}

def extract_citations(doc):

    return (
        doc.get("timesCited")
        or (doc.get("metrics") or {}).get("timesCited")
        or doc.get("citedByCount")
        or (doc.get("citations", [{}])[0].get("count") if doc.get("citations") else None)
    )

def wos_times_cited_by_doi(doi, max_retries=5):
    # WoS Starter většinou funguje s doi:"..."
    params = {"q": f'doi:"{doi}"', "limit": 1, "page": 1, "detail": "full"}
    backoff = 1.0
    for attempt in range(max_retries):
        try:
            r = requests.get(BASE_URL, headers=HEADERS, params=params, timeout=15)
            if r.status_code == 429:
                # rate limit
                time.sleep(backoff)
                backoff = min(backoff * 2, 16)
                continue
            r.raise_for_status()
            data = r.json()
            if data.get("data"):
                return extract_citations(data["data"][0])
            return None
        except requests.RequestException as e:

            if isinstance(e, requests.HTTPError) and r.status_code not in (500, 502, 503, 504):
                # špatný dotaz / nenalezeno apod.
                return None
            time.sleep(backoff)
            backoff = min(backoff * 2, 16)
    return None

results = []
for _, row in df_sample.iterrows():
    doi = str(row["doi"]).strip()
    title = row.get("title")
    if not doi or doi.lower() == "nan":
        results.append({"title": title, "doi": None, "citations": None})
        continue

    time.sleep(0.5)
    citations = wos_times_cited_by_doi(doi)
    results.append({"title": title, "doi": doi, "citations": citations})

df_results = pd.DataFrame(results)


In [ ]:
df_results.head(10)

,title,doi,citations
0,Institutions as predictors of government discr...,10.1111/kykl.12383,None
1,Determination of Tractor Engine Oil Change Int...,10.3390/ma13235403,None
2,Internet of medical things security frameworks...,10.2147/JMDH.S459987,None
3,Do the EU and Ukraine speak the same language?...,10.1080/14782804.2022.2163991,None
4,The Role of Top Management Team Digitalization...,10.3390/su12229502,None
5,How Family and a Migrant Background Influence ...,10.1177/23197145231158515,None
6,A Model of Internal Migration: An Extended Neo...,10.15240/tul/001/2020-2-004,None
7,The Role of Top Management Team Digitalization...,10.3390/su12229502,None
8,A global experiment on motivating social dista...,10.1073/pnas.2111091119,None
9,Stressing of migration matrixes for Internatio...,10.21314/JCR.2022.003,None


In [ ]:
df_file_names.head()

,file_id,file_name,file_type,cc_license,title,journal_title,journal_issn,journal_year,doi
0,19,Corruption-judicial-accountability-and-inequal...,published,none,"Corruption, judicial accountability and inequa...",Journal of Economic Behavior & Organization,0167-2681,2020,10.1016/j.jebo.2019.12.010
1,208,3-24-10-Final.pdf,accepted,none,Smart City Model Based on Systems Theory,International Journal of Information Management,0268-4012,2020,10.1016/j.ijinfomgt.2020.102092
2,213,The-evaluation-of-project-management-practices...,published,none,The Evaluation of Project Management Practices...,Economic Research-Ekonomska Istraživanja,1331-677X,2019,10.1080/1331677X.2018.1553679
3,232,Analyza.pdf,published,cc-by,Analýza vývoje československého cukrovarnictví...,Listy cukrovarnické a řepařské,1210-3306,2020,NaN
4,271,Cahlikova-Cingl-Levely-2019-How-Stress-Affects...,published,cc-by-sa,How Stress Affects Performance and Competitive...,Management Science,0025-1909,2019,10.1287/mnsc.2019.3400


In [ ]:
df_file_names.shape

(387, 9)

In [ ]:
#mergovat DOI s vystupem
# přejmenování
vystup_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/vystup.csv", sep='|')

#df_merged = pd.merge(vystup_df, df_file_names, on="", how="inner")

vystup_df.head(10)

,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,impactQbestFford,aisQbestF,aisQbestFford
0,ACTA OECONOMICA,ACTA OECONOMICA,0001-6373,0.7,0.143,Q4,ECONOMICS,Q4,ECONOMICS,Q4,50200,Q4,50200
1,ACTA VETERINARIA BRNO,ACTA VETERINARIA BRNO,0001-7213,0.7,0.106,Q3,VETERINARY SCIENCES,Q4,VETERINARY SCIENCES,Q3,40300,Q4,40300
2,ARTIFICIAL INTELLIGENCE,ARTIFICIAL INTELLIGENCE,0004-3702,4.6,1.726,Q2,"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1 (D2),"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1,10200,Q1 (D2),10200
3,BRITISH FOOD JOURNAL,BRITISH FOOD JOURNAL,0007-070X,4.2,0.531,Q1 (D2),AGRICULTURAL ECONOMICS & POLICY,Q2,AGRICULTURAL ECONOMICS & POLICY - FOOD SCIENCE...,Q1,40500 - 21100,Q2,40500 - 21100
4,CESKOSLOVENSKA PSYCHOLOGIE,CESKOSLOVENSKA PSYCHOLOGIE,0009-062X,0.4,0.128,Q4,"PSYCHOLOGY, MULTIDISCIPLINARY",Q4,"PSYCHOLOGY, MULTIDISCIPLINARY",Q4,50100,Q4,50100
5,CONVIVIUM,CONVIVIUM,0010-8235,0.1,0.267,Q4,PHILOSOPHY,Q2,PHILOSOPHY,Q4,60300,Q2,60300
6,DEMOGRAFIE,DEMOGRAFIE,0011-8265,0.6,0.098,Q4,DEMOGRAPHY,Q4,DEMOGRAPHY,Q4,50400,Q4,50400
7,EASTERN EUROPEAN ECONOMICS,EASTERN EUROPEAN ECONOMICS,0012-8775,1.2,0.249,Q3,ECONOMICS,Q3,ECONOMICS,Q3,50200,Q3,50200
8,EKONOMICKY CASOPIS,EKONOMICKY CASOPIS,0013-3035,0.6,0.073,Q4,ECONOMICS,Q4,ECONOMICS,Q4,50200,Q4,50200
9,ENVIRONMENT AND BEHAVIOR,ENVIRONMENT AND BEHAVIOR,0013-9165,4.6,1.710,Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),50700 - 50100,Q1 (D1),50700


In [ ]:
df_file_names.shape

(387, 9)

In [ ]:
df_merged = pd.merge(
    vystup_df,
    df_file_names,
    left_on='issn',
    right_on='journal_issn',
    how='inner'
)

In [ ]:
df_merged.head()

,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,...,aisQbestFford,file_id,file_name,file_type,cc_license,title,journal_title,journal_issn,journal_year,doi
0,ACTA OECONOMICA,ACTA OECONOMICA,0001-6373,0.7,0.143,Q4,ECONOMICS,Q4,ECONOMICS,Q4,...,50200,1245,pazitny-kandilaki-muzik-benkova-covid-19-cfr-a...,published,cc-by,COVID-19 Case Fatality Rate in the Context of ...,Acta Oeconomica,0001-6373,2021,10.1556/032.2021.00028
1,ACTA VETERINARIA BRNO,ACTA VETERINARIA BRNO,0001-7213,0.7,0.106,Q3,VETERINARY SCIENCES,Q4,VETERINARY SCIENCES,Q3,...,40300,713,actavet-2020089010011.pdf,published,cc-by,Intrafollicular Oocyte Transfer in Cattle – a ...,Acta Veterinaria Brno,0001-7213,2020,10.2754/avb202089010011
2,ACTA VETERINARIA BRNO,ACTA VETERINARIA BRNO,0001-7213,0.7,0.106,Q3,VETERINARY SCIENCES,Q4,VETERINARY SCIENCES,Q3,...,40300,714,actavet-2020089010003.pdf,published,cc-by,Acid-base Balance Parameters of Follicular Flu...,Acta Veterinaria Brno,0001-7213,2020,10.2754/avb202089010003
3,ARTIFICIAL INTELLIGENCE,ARTIFICIAL INTELLIGENCE,0004-3702,4.6,1.726,Q2,"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1 (D2),"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1,...,10200,842,1-s2-0-s0004370221000096-main.pdf,published,cc-by,A review of possible effects of cognitive bias...,Artificial Intelligence,0004-3702,2021,10.1016/j.artint.2021.103458
4,BRITISH FOOD JOURNAL,BRITISH FOOD JOURNAL,0007-070X,4.2,0.531,Q1 (D2),AGRICULTURAL ECONOMICS & POLICY,Q2,AGRICULTURAL ECONOMICS & POLICY - FOOD SCIENCE...,Q1,...,40500 - 21100,1116,10-1108-bfj-11-2020-1001.pdf,published,cc-by-nc-sa,Lifestyle segmentation of Czech food shoppers:...,British Food Journal,0007-070X,2021,10.1108/BFJ-11-2020-1001


In [ ]:
df_merged.shape

(314, 22)

In [ ]:
output_path = "/content/drive/My Drive/Colab Notebooks/df_merged_issn.csv"
df_merged.to_csv(output_path, index=False)

print(f"Soubor byl uložen do: {output_path}")

NameError: name 'df_merged' is not defined

In [ ]:
#abstrakt, tvůrci, nakladatele
api_key="2"

In [ ]:
test_df = df_merged_filtered_by_AIS.head(1).copy()
test_df.head()

,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,...,aisQbestFford,file_id,file_name,file_type,cc_license,title,journal_title,journal_issn,journal_year,doi
3,ARTIFICIAL INTELLIGENCE,ARTIFICIAL INTELLIGENCE,0004-3702,4.6,1.726,Q2,"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1 (D2),"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1,...,10200,842,1-s2-0-s0004370221000096-main.pdf,published,cc-by,A review of possible effects of cognitive bias...,Artificial Intelligence,0004-3702,2021,10.1016/j.artint.2021.103458


In [ ]:
# sloupce s tečkovou strukturou
llm_features_df = test_df_with_features[[col for col in test_df_with_features.columns if "." in col]]

llm_features_df.head()


,originality.incremental_score.value,impact.citations_google_scholar.value,impact.citations_web_of_science.value,impact.is_regionally_focused.value,publication.article_type.value,publication.page_length.value,publication.is_software_tool.value,publication.publisher_name.value,publication.publisher_prestige.value,publication.journal_prestige.value,authorship.total_authors.value,authorship.vse_author_count.value,authorship.vse_author_is_primary.value,authorship.collaboration_scope.value,summary.abstract_detailed.value
0,2,None,None,True,research_paper,18,False,Akadémiai Kiadó,4,3,4,2,True,2,The article examines differences in COVID‑19 c...


In [ ]:
import requests

def get_openalex_citations(doi: str):
    url = f"https://api.openalex.org/works/doi:{doi}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get("cited_by_count")
    return None

# Test:
doi = "10.1038/s41586-020-2649-2"
print(get_openalex_citations(doi))


19068


In [ ]:
df_merged_filtered_by_AIS = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/df_merged_issn.csv")
df_merged_filtered_by_AIS = df_merged_filtered_by_AIS[df_merged_filtered_by_AIS['aisQbestW'].str.contains('Q1', na=False)]


In [ ]:
df_merged_filtered_by_AIS.head(11)

,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,...,aisQbestFford,file_id,file_name,file_type,cc_license,title,journal_title,journal_issn,journal_year,doi
3,ARTIFICIAL INTELLIGENCE,ARTIFICIAL INTELLIGENCE,0004-3702,4.6,1.726,Q2,"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1 (D2),"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1,...,10200,842,1-s2-0-s0004370221000096-main.pdf,published,cc-by,A review of possible effects of cognitive bias...,Artificial Intelligence,0004-3702,2021,10.1016/j.artint.2021.103458
20,ENVIRONMENT AND BEHAVIOR,ENVIRONMENT AND BEHAVIOR,0013-9165,4.6,1.710,Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),...,50700,704,No-Evidence-of-Within-Domain-Moral-Licensing-i...,published,none,No Evidence of Within-Domain Moral Licensing i...,Environment and Behavior,0013-9165,2020,10.1177/0013916520942604
21,ENVIRONMENT AND BEHAVIOR,ENVIRONMENT AND BEHAVIOR,0013-9165,4.6,1.710,Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),...,50700,1669,pro-environmental-behavior-triggers-moral-infe...,published,none,Pro-Environmental Behavior Triggers Moral Infe...,Environment and Behavior,0013-9165,2023,10.1177/00139165231163547
29,MANAGEMENT SCIENCE,MANAGEMENT SCIENCE,0025-1909,4.9,4.180,Q1,OPERATIONS RESEARCH & MANAGEMENT SCIENCE - MAN...,Q1 (D1),OPERATIONS RESEARCH & MANAGEMENT SCIENCE - MAN...,Q1 (D2),...,50200,271,Cahlikova-Cingl-Levely-2019-How-Stress-Affects...,published,cc-by-sa,How Stress Affects Performance and Competitive...,Management Science,0025-1909,2019,10.1287/mnsc.2019.3400
70,REGIONAL STUDIES,REGIONAL STUDIES,0034-3404,4.2,1.343,Q1 (D1),ECONOMICS - GEOGRAPHY,Q1 (D2),ECONOMICS - ENVIRONMENTAL STUDIES - GEOGRAPHY ...,Q1 (D2),...,50200 - 50700,1965,eu-cohesion-policy-towards-territorial-cohesio...,published,none,EU Cohesion Policy towards territorial cohesion?,Regional Studies,0034-3404,2024,10.1080/00343404.2024.2349736
79,JOURNAL OF CONSUMER RESEARCH,JOURNAL OF CONSUMER RESEARCH,0093-5301,6.4,4.091,Q1 (D2),BUSINESS,Q1 (D1),BUSINESS,Q1 (D1),...,50200,1921,dominated-cosmopolitanism-consumer-habitus-dyn...,accepted,none,Dominated Cosmopolitanism: Consumer Habitus Dy...,Journal of Consumer Research,0093-5301,2024,10.1093/jcr/ucad035
80,SCIENTOMETRICS,SCIENTOMETRICS,0138-9130,3.5,0.814,Q1,"COMPUTER SCIENCE, INTERDISCIPLINARY APPLICATIO...",Q1,"COMPUTER SCIENCE, INTERDISCIPLINARY APPLICATIO...",Q1 (D2),...,10200 - 50800,1637,explainingcitations-cr-8-.pdf,accepted,none,Why was this cited? Explainable machine learni...,Scientometrics,0138-9130,2022,10.1007/s11192-022-04314-9
83,ENERGY ECONOMICS,ENERGY ECONOMICS,0140-9883,14.2,2.280,Q1 (D1),ECONOMICS,Q1 (D1),ECONOMICS,Q1 (D1),...,50200,1213,intolerance-predicts-climate-skepticism.pdf,published,cc-by,Intolerance predicts climate skepticism,Energy Economics,0140-9883,2022,10.1016/j.eneco.2021.105719
85,JOURNAL OF COMPARATIVE ECONOMICS,JOURNAL OF COMPARATIVE ECONOMICS,0147-5967,3.1,1.493,Q1,ECONOMICS,Q1 (D2),ECONOMICS,Q2,...,50200,1697,immigrants.pdf,published,cc-by,Immigrants from more tolerant cultures integra...,Journal of Comparative Economics,0147-5967,2023,10.1016/j.jce.2023.06.005
86,JOURNAL OF BUSINESS RESEARCH,JOURNAL OF BUSINESS RESEARCH,0148-2963,9.8,2.400,Q1 (D1),BUSINESS,Q1 (D2),BUSINESS,Q1 (D1),...,50200,1820,houdek-2024.pdf,published,none,Nudging in organizations: How to avoid behavio...,Journal of Business Research,0148-2963,2024,10.1016/j.jbusres.2024.114781


In [ ]:

df = df_merged_filtered_by_AIS.copy()
df['row_id'] = range(len(df))

# Rozdělení na 10 / 50 / 10
part1 = df.iloc[:10].copy()
part2 = df.iloc[10:60].copy()
part3 = df.iloc[60:].copy()

# Uložení do Google Drive
part1.to_csv("/content/drive/MyDrive/Colab Notebooks/part1.csv", index=False)
part2.to_csv("/content/drive/MyDrive/Colab Notebooks/part2.csv", index=False)
part3.to_csv("/content/drive/MyDrive/Colab Notebooks/part3.csv", index=False)


In [ ]:
part1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/part1.csv", sep=',')
part2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/part2.csv", sep=',')
part3 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/part3.csv", sep=',')

In [ ]:
part2.head(10)

,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,...,file_id,file_name,file_type,cc_license,title,journal_title,journal_issn,journal_year,doi,row_id
0,POLITICAL PSYCHOLOGY,POLITICAL PSYCHOLOGY,0162-895X,3.5,2.401,Q1 (D2),"POLITICAL SCIENCE - PSYCHOLOGY, SOCIAL",Q1 (D1),"POLITICAL SCIENCE - PSYCHOLOGY, SOCIAL",Q1 (D1),...,1718,political-psychology-2024-monzani-political-le...,published,cc-by-nc-nd,Political Leaders' Identity Leadership and Civ...,Political Psychology,0162-895X,2024,10.1111/pops.12952,10
1,JOURNAL OF ECONOMIC DYNAMICS & CONTROL,JOURNAL OF ECONOMIC DYNAMICS & CONTROL,0165-1889,2.3,1.153,Q2,ECONOMICS,Q1,ECONOMICS,Q2,...,687,1-s2.0-S0165188920301482-main.pdf,published,cc-by-nc-sa,"Impact of macroeconomic news, regulation and h...",Journal of Economic Dynamics and Control,0165-1889,2020,10.1016/j.jedc.2020.103980,11
2,JOURNAL OF ECONOMIC BEHAVIOR & ORGANIZATION,JOURNAL OF ECONOMIC BEHAVIOR & ORGANIZATION,0167-2681,2.3,1.272,Q2,ECONOMICS,Q1 (D2),ECONOMICS,Q2,...,19,Corruption-judicial-accountability-and-inequal...,published,none,"Corruption, judicial accountability and inequa...",Journal of Economic Behavior & Organization,0167-2681,2020,10.1016/j.jebo.2019.12.010,12
3,JOURNAL OF ECONOMIC BEHAVIOR & ORGANIZATION,JOURNAL OF ECONOMIC BEHAVIOR & ORGANIZATION,0167-2681,2.3,1.272,Q2,ECONOMICS,Q1 (D2),ECONOMICS,Q2,...,569,1-s2.0-S0167268120301505-main.pdf,published,cc-by,Understanding risk of bubbles in cryptocurrencies,Journal of Economic Behavior & Organization,0167-2681,2020,10.1016/j.jebo.2020.05.005,13
4,JOURNAL OF ECONOMIC BEHAVIOR & ORGANIZATION,JOURNAL OF ECONOMIC BEHAVIOR & ORGANIZATION,0167-2681,2.3,1.272,Q2,ECONOMICS,Q1 (D2),ECONOMICS,Q2,...,1695,globalization.pdf,published,cc-by,Does globalization suppress social trust?,Journal of Economic Behavior & Organization,0167-2681,2023,10.1016/j.jebo.2023.08.018,14
5,JOURNAL OF ECONOMIC PSYCHOLOGY,JOURNAL OF ECONOMIC PSYCHOLOGY,0167-4870,2.3,1.243,Q2,"ECONOMICS - PSYCHOLOGY, MULTIDISCIPLINARY",Q1 (D2),"ECONOMICS - PSYCHOLOGY, MULTIDISCIPLINARY",Q2,...,2160,vranka-houdek-2024-2-.pdf,published,none,Moral Hypocrisy and the Dichotomy of Hypotheti...,Journal of Economic Psychology,0167-4870,2024,10.1016/j.joep.2024.102772,15
6,INFORMATION SOCIETY,INFORMATION SOCIETY,0197-2243,3.0,1.127,Q1,COMMUNICATION - INFORMATION SCIENCE & LIBRARY ...,Q1 (D2),COMMUNICATION - INFORMATION SCIENCE & LIBRARY ...,Q1 (D2),...,1466,smutny-xxx.pdf,published,cc-by-nc-nd,"Evolution of social informatics: Publications,...",The Information Society: An International Journal,0197-2243,2022,10.1080/01972243.2022.2092570,16
7,INTERNATIONAL SMALL BUSINESS JOURNAL-RESEARCHI...,INTERNATIONAL SMALL BUSINESS JOURNAL-RESEARCHI...,0266-2426,3.9,1.448,Q2,BUSINESS - MANAGEMENT,Q1 (D2),BUSINESS - MANAGEMENT,Q1,...,1490,cowling-dvoulety-2022-uk-government-backed-sta...,published,none,UK government-backed start-up loans: Tackling ...,International Small Business Journal,0266-2426,2022,10.1177/02662426221124733,17
8,INTERNATIONAL JOURNAL OF INFORMATION MANAGEMENT,INTERNATIONAL JOURNAL OF INFORMATION MANAGEMENT,0268-4012,27.0,4.064,Q1 (D1),INFORMATION SCIENCE & LIBRARY SCIENCE,Q1 (D1),INFORMATION SCIENCE & LIBRARY SCIENCE,Q1 (D1),...,208,3-24-10-Final.pdf,accepted,none,Smart City Model Based on Systems Theory,International Journal of Information Management,0268-4012,2020,10.1016/j.ijinfomgt.2020.102092,18
9,WORLD DEVELOPMENT,WORLD DEVELOPMENT,0305-750X,4.8,1.911,Q1 (D1),DEVELOPMENT STUDIES - ECONOMICS,Q1 (D1),DEVELOPMENT STUDIES,Q1 (D2),...,2101,geography-development-and-power-parliament-lea...,published,none,"Geography, development, and power: Parliament ...",World Development,0305-750X,2024,10.1016/j.worlddev.2024.106706,19


In [ ]:
from openai import OpenAI
import os
import json
import pandas as pd
import time
from typing import Any, Dict, Optional, List



client = OpenAI(api_key="2")


def _schema_empty() -> Dict[str, Any]:
    return {
        "originality": {"incremental_score": {"value": None}},
        "impact": {
            # odstraněny citations_*;
            "is_regionally_focused": {"value": None},
        },
        "publication": {
            "article_type": {"value": None},
            "page_length": {"value": None},
            "is_software_tool": {"value": None},
            "publisher_name": {"value": None},
            "publisher_prestige": {"value": None},
            "journal_prestige": {"value": None},
        },
        "authorship": {
            "total_authors": {"value": None},
            "vse_author_count": {"value": None},
            "vse_author_is_primary": {"value": None},
            "collaboration_scope": {"value": None},
        },
        "summary": {"abstract_detailed": {"value": None}},
    }

_ALLOWED_ARTICLE_TYPES = {
    "research_paper", "review", "editorial", "brief_report", "letter", "note"
}

def _to_bool(val):
    if isinstance(val, bool):
        return val
    if isinstance(val, str):
        s = val.strip().lower()
        if s in {"true", "yes", "1"}: return True
        if s in {"false", "no", "0"}: return False
    return None

def _to_int(val, lo: Optional[int] = None, hi: Optional[int] = None):
    try:
        iv = int(float(val))
    except (TypeError, ValueError):
        return None
    if lo is not None and iv < lo: return None
    if hi is not None and iv > hi: return None
    return iv

def _sanitize_result(d: Dict[str, Any]) -> Dict[str, Any]:
    out = _schema_empty()

    # originality
    out["originality"]["incremental_score"]["value"] = _to_int(
        d.get("originality", {}).get("incremental_score", {}).get("value"),
        lo=1, hi=5
    )

    # impact (bez citations_*)
    out["impact"]["is_regionally_focused"]["value"] = _to_bool(
        d.get("impact", {}).get("is_regionally_focused", {}).get("value")
    )

    # publication
    art = d.get("publication", {}).get("article_type", {}).get("value")
    if isinstance(art, str):
        art_norm = art.strip().lower().replace(" ", "_")
        out["publication"]["article_type"]["value"] = (
            art_norm if art_norm in _ALLOWED_ARTICLE_TYPES else None
        )

    out["publication"]["page_length"]["value"] = _to_int(
        d.get("publication", {}).get("page_length", {}).get("value"), lo=1
    )
    out["publication"]["is_software_tool"]["value"] = _to_bool(
        d.get("publication", {}).get("is_software_tool", {}).get("value")
    )
    pubname = d.get("publication", {}).get("publisher_name", {}).get("value")
    out["publication"]["publisher_name"]["value"] = (
        pubname if isinstance(pubname, str) and pubname.strip() else None
    )
    out["publication"]["publisher_prestige"]["value"] = _to_int(
        d.get("publication", {}).get("publisher_prestige", {}).get("value"),
        lo=0, hi=10
    )
    out["publication"]["journal_prestige"]["value"] = _to_int(
        d.get("publication", {}).get("journal_prestige", {}).get("value"),
        lo=0, hi=10
    )

    # authorship
    out["authorship"]["total_authors"]["value"] = _to_int(
        d.get("authorship", {}).get("total_authors", {}).get("value"), lo=1
    )
    out["authorship"]["vse_author_count"]["value"] = _to_int(
        d.get("authorship", {}).get("vse_author_count", {}).get("value"), lo=0
    )
    out["authorship"]["vse_author_is_primary"]["value"] = _to_bool(
        d.get("authorship", {}).get("vse_author_is_primary", {}).get("value")
    )
    out["authorship"]["collaboration_scope"]["value"] = _to_int(
        d.get("authorship", {}).get("collaboration_scope", {}).get("value"), lo=0
    )

    # summary
    summ = d.get("summary", {}).get("abstract_detailed", {}).get("value")
    out["summary"]["abstract_detailed"]["value"] = (
        summ if isinstance(summ, str) and summ.strip() else None
    )

    return out

def _flatten_to_columns_clean_underscores(d: Dict[str, Any], prefix: str = "") -> Dict[str, Any]:
    """
    Rekurzivně projde dict a uloží jen položky {"value": ...} do plochých názvů sloupců:
    - bez teček
    - bez suffixu .value
    - názvy poskládá podtržítky, např. publication_page_length
    """
    flat: Dict[str, Any] = {}
    if not isinstance(d, dict):
        return flat

    for k, v in d.items():
        key = f"{prefix}_{k}" if prefix else k
        if isinstance(v, dict):
            if set(v.keys()) == {"value"}:
                flat[key] = v["value"]   # <<< žádné _value na konci
            else:
                flat.update(_flatten_to_columns_clean_underscores(v, key))
    return flat

# ---------- Dotaz na LLM ----------
def _extract_json(text: str) -> Optional[Dict[str, Any]]:
    """
    Zkusí vytáhnout JSON: ořízne code fences a načte JSON.
    """
    if not isinstance(text, str):
        return None
    s = text.strip()
    if s.startswith("```"):
        s = s.strip("`")
        if s.lstrip().startswith("json"):
            s = s.lstrip()[4:].strip()
    try:
        return json.loads(s)
    except Exception:
        return None

def get_doi_features(doi: str) -> Dict[str, Any]:
    if not doi or not isinstance(doi, str):
        return _schema_empty()

    prompt = f"""
Jsi pečlivý bibliometrický asistent s přístupem k webovému vyhledávání.
Dostal jsi DOI: {doi}

ÚKOLY:
1) Najdi oficiální záznam článku (stránka vydavatele/časopisu), Crossref a další autoritativní zdroje pomocí webového vyhledávání.
2) Abstrakt vědeckého článku je určen k citování a JE LEGÁLNÍ ho použít. Pokud je dostupný, normálně s ním pracuj (NEODMÍTEJ kvůli autorským právům).
3) Požadované položky vyplň jen na základě dohledatelných údajů. Pokud se údaj nedá najít nebo si nejsi jistý, vrať přesně null.
4) Vyplň POUZE následující položky (ostatní ignoruj):

- originality.incremental_score (1–5, integer)
- impact.is_regionally_focused (boolean)
- publication.article_type (enum: ["research_paper","review","editorial","brief_report","letter","note"])
- publication.page_length (integer)
- publication.is_software_tool (boolean)
- publication.publisher_name (string)
- publication.publisher_prestige (0–10, integer)
- publication.journal_prestige (0–10, integer)
- authorship.total_authors (integer)
- authorship.vse_author_count (integer)
- authorship.vse_author_is_primary (boolean)
- authorship.collaboration_scope (integer; počet unikátních zemí v afilacích autorů)
- summary.abstract_detailed (string)

VRAŤ JEN platný JSON dle schématu (žádný text navíc):

{{
  "originality": {{
    "incremental_score": {{ "value": <integer 1-5 or null> }}
  }},
  "impact": {{
    "is_regionally_focused": {{ "value": <true|false or null> }}
  }},
  "publication": {{
    "article_type": {{ "value": <"research_paper"|"review"|"editorial"|"brief_report"|"letter"|"note" or null> }},
    "page_length": {{ "value": <integer or null> }},
    "is_software_tool": {{ "value": <true|false or null> }},
    "publisher_name": {{ "value": <string or null> }},
    "publisher_prestige": {{ "value": <integer 0-10 or null> }},
    "journal_prestige": {{ "value": <integer 0-10 or null> }}
  }},
  "authorship": {{
    "total_authors": {{ "value": <integer or null> }},
    "vse_author_count": {{ "value": <integer or null> }},
    "vse_author_is_primary": {{ "value": <true|false or null> }},
    "collaboration_scope": {{ "value": <integer or null> }}
  }},
  "summary": {{
    "abstract_detailed": {{ "value": <string or null> }}
  }}
}}

DEFINICE:
- VŠE = "Vysoká škola ekonomická v Praze" = "Prague University of Economics and Business" = "University of Economics, Prague" = "VSE".
- "vse_author_is_primary" = true, pokud je autor z VŠE prvním NEBO korespondujícím autorem.
- "is_regionally_focused" = true, pokud je úzký regionální/tematický fokus s omezenou mezinárodní relevancí.
- Pokud údaj nenajdeš nebo je nejasný, vrať null.
"""
    try:
        resp = client.responses.create(
            model="gpt-5-mini",
            tools=[{"type": "web_search"}],  # web_search ponechán
            input=prompt
        )
        raw = getattr(resp, "output_text", None)
        data = _extract_json(raw) if raw else None
        if not isinstance(data, dict):
            return _schema_empty()
    except Exception:
        return _schema_empty()

    return _sanitize_result(data)

# ---------- Propojení s DataFrame ----------
def enrich_df_with_llm_features(
    df: pd.DataFrame,
    doi_col: str = "doi",
    sleep_s: float = 0.0,
    output_csv: Optional[str] = "merged_df_with_LLM_features.csv",
    retries: int = 2,
) -> pd.DataFrame:
    """
    Pro každý DOI ve sloupci `doi_col` zavolá get_doi_features a přidá nové sloupce.
    Výstupní názvy sloupců jsou bez teček i bez suffixu .value, např.:
      originality_incremental_score, publication_page_length, summary_abstract_detailed, ...
    """
    if doi_col not in df.columns:
        raise ValueError(f"V DataFrame chybí sloupec '{doi_col}'.")

    results: List[Dict[str, Any]] = []
    for doi in df[doi_col].fillna(""):
        if not isinstance(doi, str) or not doi.strip():
            results.append(_schema_empty())
            continue

        attempt = 0
        payload = None
        while attempt <= retries:
            payload = get_doi_features(doi.strip())
            if payload != _schema_empty():
                break
            attempt += 1
            if sleep_s > 0:
                time.sleep(sleep_s)

        results.append(payload or _schema_empty())
        if sleep_s > 0:
            time.sleep(sleep_s)

    # Rozbalit do sloupců s podtržítky a bez .value
    flat_rows = [_flatten_to_columns_clean_underscores(r) for r in results]
    feats_df = pd.DataFrame(flat_rows)
    df_out = df.reset_index(drop=True).join(feats_df)

    if output_csv:
        df_out.to_csv(output_csv, index=False, encoding="utf-8-sig")
    return df_out



part3_with_features = enrich_df_with_llm_features(
    part3,
    doi_col="doi",
    sleep_s=0.0,
    #output_csv="/content/drive/MyDrive/Colab Notebooks/part1_with_LLM.csv"
)
part3_with_features.to_csv("/content/drive/MyDrive/Colab Notebooks/part3_with_features.csv", index=False)
part3_with_features.head(10)


,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,...,publication_page_length,publication_is_software_tool,publication_publisher_name,publication_publisher_prestige,publication_journal_prestige,authorship_total_authors,authorship_vse_author_count,authorship_vse_author_is_primary,authorship_collaboration_scope,summary_abstract_detailed
0,FASHION PRACTICE-THE JOURNAL OF DESIGN CREATIV...,FASHION PRACTICE-THE JOURNAL OF DESIGN CREATIV...,1756-9370,2.2,0.366,Q1 (D1),"HUMANITIES, MULTIDISCIPLINARY",Q1 (D2),"HUMANITIES, MULTIDISCIPLINARY",Q1 (D1),...,None,None,None,None,None,None,None,None,None,None
1,INTERNATIONAL JOURNAL OF CENTRAL BANKING,INTERNATIONAL JOURNAL OF CENTRAL BANKING,1815-4654,1.5,1.239,Q3,"BUSINESS, FINANCE",Q1 (D2),"BUSINESS, FINANCE",Q3,...,None,None,None,None,None,None,None,None,None,None
2,SOCIAL PSYCHOLOGICAL AND PERSONALITY SCIENCE,SOCIAL PSYCHOLOGICAL AND PERSONALITY SCIENCE,1948-5506,3.3,2.133,Q1 (D2),"PSYCHOLOGY, SOCIAL",Q1 (D2),"PSYCHOLOGY, SOCIAL",Q1 (D2),...,None,None,None,None,None,None,None,None,None,None
3,SCIENTIFIC REPORTS,SCIENTIFIC REPORTS,2045-2322,3.9,1.029,Q1 (D2),MULTIDISCIPLINARY SCIENCES,Q1,MULTIDISCIPLINARY SCIENCES,Q1 (D2),...,None,None,None,None,None,None,None,None,None,None
4,NUTRIENTS,NUTRIENTS,2072-6643,5.0,1.187,Q1 (D2),NUTRITION & DIETETICS,Q1,NUTRITION & DIETETICS,Q1 (D1),...,None,None,None,None,None,None,None,None,None,None
5,AGRONOMY-BASEL,AGRONOMY-BASEL,2073-4395,3.4,0.544,Q1 (D2),AGRONOMY,Q1 (D2),AGRONOMY,Q1 (D2),...,None,None,None,None,None,None,None,None,None,None
6,TOURISM MANAGEMENT PERSPECTIVES,TOURISM MANAGEMENT PERSPECTIVES,2211-9736,6.9,1.404,Q1 (D2),"HOSPITALITY, LEISURE, SPORT & TOURISM - MANAGE...",Q1,"HOSPITALITY, LEISURE, SPORT & TOURISM - MANAGE...",Q1 (D1),...,None,None,None,None,None,None,None,None,None,None
7,JOURNAL OF THE ASSOCIATION FOR INFORMATION SCI...,JOURNAL OF THE ASSOCIATION FOR INFORMATION SCI...,2330-1635,4.3,0.942,Q1 (D2),"COMPUTER SCIENCE, INFORMATION SYSTEMS - INFORM...",Q1 (D2),"COMPUTER SCIENCE, INFORMATION SYSTEMS - INFORM...",Q1 (D1),...,None,None,None,None,None,None,None,None,None,None
8,NATURE HUMAN BEHAVIOUR,NATURE HUMAN BEHAVIOUR,2397-3374,15.9,9.102,Q1 (D1),MULTIDISCIPLINARY SCIENCES - NEUROSCIENCES - P...,Q1 (D1),MULTIDISCIPLINARY SCIENCES - NEUROSCIENCES - P...,Q1 (D1),...,None,None,None,None,None,None,None,None,None,None
9,NATURE HUMAN BEHAVIOUR,NATURE HUMAN BEHAVIOUR,2397-3374,15.9,9.102,Q1 (D1),MULTIDISCIPLINARY SCIENCES - NEUROSCIENCES - P...,Q1 (D1),MULTIDISCIPLINARY SCIENCES - NEUROSCIENCES - P...,Q1 (D1),...,None,None,None,None,None,None,None,None,None,None


In [ ]:
part3_with_features.head(11)

,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,...,publication_page_length,publication_is_software_tool,publication_publisher_name,publication_publisher_prestige,publication_journal_prestige,authorship_total_authors,authorship_vse_author_count,authorship_vse_author_is_primary,authorship_collaboration_scope,summary_abstract_detailed
0,FASHION PRACTICE-THE JOURNAL OF DESIGN CREATIV...,FASHION PRACTICE-THE JOURNAL OF DESIGN CREATIV...,1756-9370,2.2,0.366,Q1 (D1),"HUMANITIES, MULTIDISCIPLINARY",Q1 (D2),"HUMANITIES, MULTIDISCIPLINARY",Q1 (D1),...,None,None,None,None,None,None,None,None,None,None
1,INTERNATIONAL JOURNAL OF CENTRAL BANKING,INTERNATIONAL JOURNAL OF CENTRAL BANKING,1815-4654,1.5,1.239,Q3,"BUSINESS, FINANCE",Q1 (D2),"BUSINESS, FINANCE",Q3,...,None,None,None,None,None,None,None,None,None,None
2,SOCIAL PSYCHOLOGICAL AND PERSONALITY SCIENCE,SOCIAL PSYCHOLOGICAL AND PERSONALITY SCIENCE,1948-5506,3.3,2.133,Q1 (D2),"PSYCHOLOGY, SOCIAL",Q1 (D2),"PSYCHOLOGY, SOCIAL",Q1 (D2),...,None,None,None,None,None,None,None,None,None,None
3,SCIENTIFIC REPORTS,SCIENTIFIC REPORTS,2045-2322,3.9,1.029,Q1 (D2),MULTIDISCIPLINARY SCIENCES,Q1,MULTIDISCIPLINARY SCIENCES,Q1 (D2),...,None,None,None,None,None,None,None,None,None,None
4,NUTRIENTS,NUTRIENTS,2072-6643,5.0,1.187,Q1 (D2),NUTRITION & DIETETICS,Q1,NUTRITION & DIETETICS,Q1 (D1),...,None,None,None,None,None,None,None,None,None,None
5,AGRONOMY-BASEL,AGRONOMY-BASEL,2073-4395,3.4,0.544,Q1 (D2),AGRONOMY,Q1 (D2),AGRONOMY,Q1 (D2),...,None,None,None,None,None,None,None,None,None,None
6,TOURISM MANAGEMENT PERSPECTIVES,TOURISM MANAGEMENT PERSPECTIVES,2211-9736,6.9,1.404,Q1 (D2),"HOSPITALITY, LEISURE, SPORT & TOURISM - MANAGE...",Q1,"HOSPITALITY, LEISURE, SPORT & TOURISM - MANAGE...",Q1 (D1),...,None,None,None,None,None,None,None,None,None,None
7,JOURNAL OF THE ASSOCIATION FOR INFORMATION SCI...,JOURNAL OF THE ASSOCIATION FOR INFORMATION SCI...,2330-1635,4.3,0.942,Q1 (D2),"COMPUTER SCIENCE, INFORMATION SYSTEMS - INFORM...",Q1 (D2),"COMPUTER SCIENCE, INFORMATION SYSTEMS - INFORM...",Q1 (D1),...,None,None,None,None,None,None,None,None,None,None
8,NATURE HUMAN BEHAVIOUR,NATURE HUMAN BEHAVIOUR,2397-3374,15.9,9.102,Q1 (D1),MULTIDISCIPLINARY SCIENCES - NEUROSCIENCES - P...,Q1 (D1),MULTIDISCIPLINARY SCIENCES - NEUROSCIENCES - P...,Q1 (D1),...,None,None,None,None,None,None,None,None,None,None
9,NATURE HUMAN BEHAVIOUR,NATURE HUMAN BEHAVIOUR,2397-3374,15.9,9.102,Q1 (D1),MULTIDISCIPLINARY SCIENCES - NEUROSCIENCES - P...,Q1 (D1),MULTIDISCIPLINARY SCIENCES - NEUROSCIENCES - P...,Q1 (D1),...,None,None,None,None,None,None,None,None,None,None


In [ ]:
part1_with_features = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/part1_with_features.csv")
part2_with_features = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/part2_with_features.csv")
part3_with_features = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/part3_with_features.csv")

In [ ]:
part3_with_features.shape

(10, 36)

In [ ]:
combined_df = pd.concat(
    [part1_with_features, part2_with_features, part3_with_features],
    ignore_index=True
)
combined_df.head()

,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,...,publication_page_length,publication_is_software_tool,publication_publisher_name,publication_publisher_prestige,publication_journal_prestige,authorship_total_authors,authorship_vse_author_count,authorship_vse_author_is_primary,authorship_collaboration_scope,summary_abstract_detailed
0,ARTIFICIAL INTELLIGENCE,ARTIFICIAL INTELLIGENCE,0004-3702,4.6,1.726,Q2,"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1 (D2),"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1,...,33.0,False,Elsevier,9.0,9.0,3.0,2.0,True,2.0,While the interpretability of machine learning...
1,ENVIRONMENT AND BEHAVIOR,ENVIRONMENT AND BEHAVIOR,0013-9165,4.6,1.710,Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),...,25.0,False,SAGE Publications,8.0,7.0,3.0,1.0,False,1.0,Several studies have suggested that people mig...
2,ENVIRONMENT AND BEHAVIOR,ENVIRONMENT AND BEHAVIOR,0013-9165,4.6,1.710,Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),...,25.0,False,SAGE Publications,NaN,NaN,3.0,1.0,False,1.0,Several studies have shown that moral licensin...
3,MANAGEMENT SCIENCE,MANAGEMENT SCIENCE,0025-1909,4.9,4.180,Q1,OPERATIONS RESEARCH & MANAGEMENT SCIENCE - MAN...,Q1 (D1),OPERATIONS RESEARCH & MANAGEMENT SCIENCE - MAN...,Q1 (D2),...,16.0,False,Institute for Operations Research and the Mana...,NaN,NaN,3.0,1.0,False,3.0,The authors use a laboratory experiment that i...
4,REGIONAL STUDIES,REGIONAL STUDIES,0034-3404,4.2,1.343,Q1 (D1),ECONOMICS - GEOGRAPHY,Q1 (D2),ECONOMICS - ENVIRONMENTAL STUDIES - GEOGRAPHY ...,Q1 (D2),...,5.0,False,"Routledge Journals, Taylor & Francis Ltd",NaN,NaN,4.0,0.0,False,4.0,Despite being one of the main European Union (...


In [ ]:
combined_df.shape

(70, 36)

In [ ]:
import requests

def get_openalex_citations(doi: str):
    url = f"https://api.openalex.org/works/doi:{doi}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get("cited_by_count")
    return None




In [ ]:
combined_df["citations_openalex"] = combined_df["doi"].apply(get_openalex_citations)

In [ ]:
combined_df.head()

,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,...,publication_is_software_tool,publication_publisher_name,publication_publisher_prestige,publication_journal_prestige,authorship_total_authors,authorship_vse_author_count,authorship_vse_author_is_primary,authorship_collaboration_scope,summary_abstract_detailed,citations_openalex
0,ARTIFICIAL INTELLIGENCE,ARTIFICIAL INTELLIGENCE,0004-3702,4.6,1.726,Q2,"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1 (D2),"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1,...,False,Elsevier,9.0,9.0,3.0,2.0,True,2.0,While the interpretability of machine learning...,106.0
1,ENVIRONMENT AND BEHAVIOR,ENVIRONMENT AND BEHAVIOR,0013-9165,4.6,1.710,Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),...,False,SAGE Publications,8.0,7.0,3.0,1.0,False,1.0,Several studies have suggested that people mig...,26.0
2,ENVIRONMENT AND BEHAVIOR,ENVIRONMENT AND BEHAVIOR,0013-9165,4.6,1.710,Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),...,False,SAGE Publications,NaN,NaN,3.0,1.0,False,1.0,Several studies have shown that moral licensin...,8.0
3,MANAGEMENT SCIENCE,MANAGEMENT SCIENCE,0025-1909,4.9,4.180,Q1,OPERATIONS RESEARCH & MANAGEMENT SCIENCE - MAN...,Q1 (D1),OPERATIONS RESEARCH & MANAGEMENT SCIENCE - MAN...,Q1 (D2),...,False,Institute for Operations Research and the Mana...,NaN,NaN,3.0,1.0,False,3.0,The authors use a laboratory experiment that i...,64.0
4,REGIONAL STUDIES,REGIONAL STUDIES,0034-3404,4.2,1.343,Q1 (D1),ECONOMICS - GEOGRAPHY,Q1 (D2),ECONOMICS - ENVIRONMENTAL STUDIES - GEOGRAPHY ...,Q1 (D2),...,False,"Routledge Journals, Taylor & Francis Ltd",NaN,NaN,4.0,0.0,False,4.0,Despite being one of the main European Union (...,3.0


In [ ]:
combined_df.shape

(70, 37)

In [ ]:
combined_df.to_csv("/content/drive/MyDrive/Colab Notebooks/df_LLM_features.csv", index=False)


In [ ]:
combined_df.tail(24)

,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,...,publication_is_software_tool,publication_publisher_name,publication_publisher_prestige,publication_journal_prestige,authorship_total_authors,authorship_vse_author_count,authorship_vse_author_is_primary,authorship_collaboration_scope,summary_abstract_detailed,citations_openalex
46,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,1472-8117,7.4,0.929,Q1 (D1),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH - ...,Q1 (D2),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH,Q1 (D1),...,False,Elsevier,NaN,NaN,1.0,1.0,True,1.0,NaN,4.0
47,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,1472-8117,7.4,0.929,Q1 (D1),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH - ...,Q1 (D2),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH,Q1 (D1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0
48,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,1472-8117,7.4,0.929,Q1 (D1),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH - ...,Q1 (D2),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH,Q1 (D1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
49,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,1472-8117,7.4,0.929,Q1 (D1),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH - ...,Q1 (D2),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH,Q1 (D1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
50,JOURNAL OF CONTEMPORARY EUROPEAN STUDIES,JOURNAL OF CONTEMPORARY EUROPEAN STUDIES,1478-2804,1.8,0.613,Q1 (D1),AREA STUDIES,Q1,AREA STUDIES,Q2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
51,HUMAN RESOURCES FOR HEALTH,HUMAN RESOURCES FOR HEALTH,1478-4491,4.3,1.311,Q1 (D1),HEALTH POLICY & SERVICES - INDUSTRIAL RELATION...,Q1 (D2),HEALTH POLICY & SERVICES - INDUSTRIAL RELATION...,Q1 (D1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
52,CORPORATE SOCIAL RESPONSIBILITY AND ENVIRONMEN...,CORPORATE SOCIAL RESPONSIBILITY AND ENVIRONMEN...,1535-3958,9.1,1.520,Q1 (D1),BUSINESS - ENVIRONMENTAL STUDIES - MANAGEMENT,Q1 (D2),BUSINESS - ENVIRONMENTAL STUDIES - MANAGEMENT,Q1 (D1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0
53,JOURNAL OF THE EUROPEAN ECONOMIC ASSOCIATION,JOURNAL OF THE EUROPEAN ECONOMIC ASSOCIATION,1542-4766,3.3,5.766,Q1 (D2),ECONOMICS,Q1 (D1),ECONOMICS,Q2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
54,FINANCE RESEARCH LETTERS,FINANCE RESEARCH LETTERS,1544-6123,6.9,1.076,Q1 (D1),"BUSINESS, FINANCE",Q1 (D2),"BUSINESS, FINANCE",Q1 (D1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
55,FINANCE RESEARCH LETTERS,FINANCE RESEARCH LETTERS,1544-6123,6.9,1.076,Q1 (D1),"BUSINESS, FINANCE",Q1 (D2),"BUSINESS, FINANCE",Q1 (D1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [ ]:
missing_df = combined_df.tail(23).copy()

In [ ]:
missing_df.head(30)

,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,...,publication_is_software_tool,publication_publisher_name,publication_publisher_prestige,publication_journal_prestige,authorship_total_authors,authorship_vse_author_count,authorship_vse_author_is_primary,authorship_collaboration_scope,summary_abstract_detailed,citations_openalex
47,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,1472-8117,7.4,0.929,Q1 (D1),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH - ...,Q1 (D2),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH,Q1 (D1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0
48,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,1472-8117,7.4,0.929,Q1 (D1),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH - ...,Q1 (D2),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH,Q1 (D1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
49,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,1472-8117,7.4,0.929,Q1 (D1),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH - ...,Q1 (D2),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH,Q1 (D1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
50,JOURNAL OF CONTEMPORARY EUROPEAN STUDIES,JOURNAL OF CONTEMPORARY EUROPEAN STUDIES,1478-2804,1.8,0.613,Q1 (D1),AREA STUDIES,Q1,AREA STUDIES,Q2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
51,HUMAN RESOURCES FOR HEALTH,HUMAN RESOURCES FOR HEALTH,1478-4491,4.3,1.311,Q1 (D1),HEALTH POLICY & SERVICES - INDUSTRIAL RELATION...,Q1 (D2),HEALTH POLICY & SERVICES - INDUSTRIAL RELATION...,Q1 (D1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
52,CORPORATE SOCIAL RESPONSIBILITY AND ENVIRONMEN...,CORPORATE SOCIAL RESPONSIBILITY AND ENVIRONMEN...,1535-3958,9.1,1.520,Q1 (D1),BUSINESS - ENVIRONMENTAL STUDIES - MANAGEMENT,Q1 (D2),BUSINESS - ENVIRONMENTAL STUDIES - MANAGEMENT,Q1 (D1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0
53,JOURNAL OF THE EUROPEAN ECONOMIC ASSOCIATION,JOURNAL OF THE EUROPEAN ECONOMIC ASSOCIATION,1542-4766,3.3,5.766,Q1 (D2),ECONOMICS,Q1 (D1),ECONOMICS,Q2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
54,FINANCE RESEARCH LETTERS,FINANCE RESEARCH LETTERS,1544-6123,6.9,1.076,Q1 (D1),"BUSINESS, FINANCE",Q1 (D2),"BUSINESS, FINANCE",Q1 (D1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
55,FINANCE RESEARCH LETTERS,FINANCE RESEARCH LETTERS,1544-6123,6.9,1.076,Q1 (D1),"BUSINESS, FINANCE",Q1 (D2),"BUSINESS, FINANCE",Q1 (D1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
56,FINANCE RESEARCH LETTERS,FINANCE RESEARCH LETTERS,1544-6123,6.9,1.076,Q1 (D1),"BUSINESS, FINANCE",Q1 (D2),"BUSINESS, FINANCE",Q1 (D1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0


In [ ]:
missing_df_clean = missing_df[part1.columns].copy()

In [ ]:
missing_df_clean.to_csv("/content/drive/MyDrive/Colab Notebooks/missing_df_clean.csv", index=False)


In [ ]:
missing_df_clean.shape

(23, 23)

In [ ]:
test_df_2 = missing_df_clean.head(1).copy()
test_df_2.head()

,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,...,file_id,file_name,file_type,cc_license,title,journal_title,journal_issn,journal_year,doi,row_id
47,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,1472-8117,7.4,0.929,Q1 (D1),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH - ...,Q1 (D2),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH,Q1 (D1),...,1717,houdek-2023-1-.pdf,published,none,The deep roots of cross-cultural differences i...,The International Journal of Management Education,1472-8117,2023,10.1016/j.ijme.2023.100876,47


In [ ]:
from openai import OpenAI
import os
import json
import pandas as pd
import time
from typing import Any, Dict, Optional, List


client = OpenAI(api_key="2")


def _schema_empty() -> Dict[str, Any]:
    return {
        "originality": {"incremental_score": {"value": None}},
        "impact": {
            # odstraněny citations_*; ponecháno jen:
            "is_regionally_focused": {"value": None},
        },
        "publication": {
            "article_type": {"value": None},
            "page_length": {"value": None},
            "is_software_tool": {"value": None},
            "publisher_name": {"value": None},
            "publisher_prestige": {"value": None},
            "journal_prestige": {"value": None},
        },
        "authorship": {
            "total_authors": {"value": None},
            "vse_author_count": {"value": None},
            "vse_author_is_primary": {"value": None},
            "collaboration_scope": {"value": None},
        },
        "summary": {"abstract_detailed": {"value": None}},
    }

_ALLOWED_ARTICLE_TYPES = {
    "research_paper", "review", "editorial", "brief_report", "letter", "note"
}

def _to_bool(val):
    if isinstance(val, bool):
        return val
    if isinstance(val, str):
        s = val.strip().lower()
        if s in {"true", "yes", "1"}: return True
        if s in {"false", "no", "0"}: return False
    return None

def _to_int(val, lo: Optional[int] = None, hi: Optional[int] = None):
    try:
        iv = int(float(val))
    except (TypeError, ValueError):
        return None
    if lo is not None and iv < lo: return None
    if hi is not None and iv > hi: return None
    return iv

def _sanitize_result(d: Dict[str, Any]) -> Dict[str, Any]:
    out = _schema_empty()

    # originality
    out["originality"]["incremental_score"]["value"] = _to_int(
        d.get("originality", {}).get("incremental_score", {}).get("value"),
        lo=1, hi=5
    )

    # impact (bez citations_*)
    out["impact"]["is_regionally_focused"]["value"] = _to_bool(
        d.get("impact", {}).get("is_regionally_focused", {}).get("value")
    )

    # publication
    art = d.get("publication", {}).get("article_type", {}).get("value")
    if isinstance(art, str):
        art_norm = art.strip().lower().replace(" ", "_")
        out["publication"]["article_type"]["value"] = (
            art_norm if art_norm in _ALLOWED_ARTICLE_TYPES else None
        )

    out["publication"]["page_length"]["value"] = _to_int(
        d.get("publication", {}).get("page_length", {}).get("value"), lo=1
    )
    out["publication"]["is_software_tool"]["value"] = _to_bool(
        d.get("publication", {}).get("is_software_tool", {}).get("value")
    )
    pubname = d.get("publication", {}).get("publisher_name", {}).get("value")
    out["publication"]["publisher_name"]["value"] = (
        pubname if isinstance(pubname, str) and pubname.strip() else None
    )
    out["publication"]["publisher_prestige"]["value"] = _to_int(
        d.get("publication", {}).get("publisher_prestige", {}).get("value"),
        lo=0, hi=10
    )
    out["publication"]["journal_prestige"]["value"] = _to_int(
        d.get("publication", {}).get("journal_prestige", {}).get("value"),
        lo=0, hi=10
    )

    # authorship
    out["authorship"]["total_authors"]["value"] = _to_int(
        d.get("authorship", {}).get("total_authors", {}).get("value"), lo=1
    )
    out["authorship"]["vse_author_count"]["value"] = _to_int(
        d.get("authorship", {}).get("vse_author_count", {}).get("value"), lo=0
    )
    out["authorship"]["vse_author_is_primary"]["value"] = _to_bool(
        d.get("authorship", {}).get("vse_author_is_primary", {}).get("value")
    )
    out["authorship"]["collaboration_scope"]["value"] = _to_int(
        d.get("authorship", {}).get("collaboration_scope", {}).get("value"), lo=0
    )

    # summary
    summ = d.get("summary", {}).get("abstract_detailed", {}).get("value")
    out["summary"]["abstract_detailed"]["value"] = (
        summ if isinstance(summ, str) and summ.strip() else None
    )

    return out

def _flatten_to_columns_clean_underscores(d: Dict[str, Any], prefix: str = "") -> Dict[str, Any]:
    """
    Rekurzivně projde dict a uloží jen položky {"value": ...} do plochých názvů sloupců:
    - bez teček
    - bez suffixu .value
    - názvy poskládá podtržítky, např. publication_page_length
    """
    flat: Dict[str, Any] = {}
    if not isinstance(d, dict):
        return flat

    for k, v in d.items():
        key = f"{prefix}_{k}" if prefix else k
        if isinstance(v, dict):
            if set(v.keys()) == {"value"}:
                flat[key] = v["value"]   # <<< žádné _value na konci
            else:
                flat.update(_flatten_to_columns_clean_underscores(v, key))
    return flat

# ---------- Dotaz na LLM ----------
def _extract_json(text: str) -> Optional[Dict[str, Any]]:
    """
    Zkusí vytáhnout JSON: ořízne code fences a načte JSON.
    """
    if not isinstance(text, str):
        return None
    s = text.strip()
    if s.startswith("```"):
        s = s.strip("`")
        if s.lstrip().startswith("json"):
            s = s.lstrip()[4:].strip()
    try:
        return json.loads(s)
    except Exception:
        return None

def get_doi_features(doi: str) -> Dict[str, Any]:
    if not doi or not isinstance(doi, str):
        return _schema_empty()

    prompt = f"""
Jsi pečlivý bibliometrický asistent s přístupem k webovému vyhledávání.
Dostal jsi DOI: {doi}

ÚKOLY:
1) Najdi oficiální záznam článku (stránka vydavatele/časopisu), Crossref a další autoritativní zdroje pomocí webového vyhledávání.
2) Abstrakt vědeckého článku je určen k citování a JE LEGÁLNÍ ho použít. Pokud je dostupný, normálně s ním pracuj (NEODMÍTEJ kvůli autorským právům).
3) Požadované položky vyplň jen na základě dohledatelných údajů. Pokud se údaj nedá najít nebo si nejsi jistý, vrať přesně null.
4) Vyplň POUZE následující položky (ostatní ignoruj):

- originality.incremental_score (1–5, integer)
- impact.is_regionally_focused (boolean)
- publication.article_type (enum: ["research_paper","review","editorial","brief_report","letter","note"])
- publication.page_length (integer)
- publication.is_software_tool (boolean)
- publication.publisher_name (string)
- publication.publisher_prestige (0–10, integer)
- publication.journal_prestige (0–10, integer)
- authorship.total_authors (integer)
- authorship.vse_author_count (integer)
- authorship.vse_author_is_primary (boolean)
- authorship.collaboration_scope (integer; počet unikátních zemí v afilacích autorů)
- summary.abstract_detailed (string)

VRAŤ JEN platný JSON dle schématu (žádný text navíc):

{{
  "originality": {{
    "incremental_score": {{ "value": <integer 1-5 or null> }}
  }},
  "impact": {{
    "is_regionally_focused": {{ "value": <true|false or null> }}
  }},
  "publication": {{
    "article_type": {{ "value": <"research_paper"|"review"|"editorial"|"brief_report"|"letter"|"note" or null> }},
    "page_length": {{ "value": <integer or null> }},
    "is_software_tool": {{ "value": <true|false or null> }},
    "publisher_name": {{ "value": <string or null> }},
    "publisher_prestige": {{ "value": <integer 0-10 or null> }},
    "journal_prestige": {{ "value": <integer 0-10 or null> }}
  }},
  "authorship": {{
    "total_authors": {{ "value": <integer or null> }},
    "vse_author_count": {{ "value": <integer or null> }},
    "vse_author_is_primary": {{ "value": <true|false or null> }},
    "collaboration_scope": {{ "value": <integer or null> }}
  }},
  "summary": {{
    "abstract_detailed": {{ "value": <string or null> }}
  }}
}}

DEFINICE:
- VŠE = "Vysoká škola ekonomická v Praze" = "Prague University of Economics and Business" = "University of Economics, Prague" = "VSE".
- "vse_author_is_primary" = true, pokud je autor z VŠE prvním NEBO korespondujícím autorem.
- "is_regionally_focused" = true, pokud je úzký regionální/tematický fokus s omezenou mezinárodní relevancí.
- Pokud údaj nenajdeš nebo je nejasný, vrať null.
"""
    try:
        resp = client.responses.create(
            model="gpt-5-mini",
            tools=[{"type": "web_search"}],  # web_search ponechán
            input=prompt
        )
        raw = getattr(resp, "output_text", None)
        data = _extract_json(raw) if raw else None
        if not isinstance(data, dict):
            return _schema_empty()
    except Exception:
        return _schema_empty()

    return _sanitize_result(data)

# ---------- Propojení s DataFrame ----------
def enrich_df_with_llm_features(
    df: pd.DataFrame,
    doi_col: str = "doi",
    sleep_s: float = 0.0,
    output_csv: Optional[str] = "merged_df_with_LLM_features.csv",
    retries: int = 2,
) -> pd.DataFrame:
    """
    Pro každý DOI ve sloupci `doi_col` zavolá get_doi_features a přidá nové sloupce.
    Výstupní názvy sloupců jsou bez teček i bez suffixu .value, např.:
      originality_incremental_score, publication_page_length, summary_abstract_detailed, ...
    """
    if doi_col not in df.columns:
        raise ValueError(f"V DataFrame chybí sloupec '{doi_col}'.")

    results: List[Dict[str, Any]] = []
    for doi in df[doi_col].fillna(""):
        if not isinstance(doi, str) or not doi.strip():
            results.append(_schema_empty())
            continue

        attempt = 0
        payload = None
        while attempt <= retries:
            payload = get_doi_features(doi.strip())
            if payload != _schema_empty():
                break
            attempt += 1
            if sleep_s > 0:
                time.sleep(sleep_s)

        results.append(payload or _schema_empty())
        if sleep_s > 0:
            time.sleep(sleep_s)

    # Rozbalit do sloupců s podtržítky a bez .value
    flat_rows = [_flatten_to_columns_clean_underscores(r) for r in results]
    feats_df = pd.DataFrame(flat_rows)
    df_out = df.reset_index(drop=True).join(feats_df)

    if output_csv:
        df_out.to_csv(output_csv, index=False, encoding="utf-8-sig")
    return df_out



missing_df_with_features = enrich_df_with_llm_features(
    missing_df_clean,
    doi_col="doi",
    sleep_s=0.0,
    #output_csv="/content/drive/MyDrive/Colab Notebooks/part1_with_LLM.csv"
)
missing_df_with_features.to_csv("/content/drive/MyDrive/Colab Notebooks/missing_df_features.csv", index=False)
#test_df_2_with_features.head(10)


In [ ]:
combined_df_trimmed = combined_df.iloc[:-23].copy()


In [ ]:
combined_df_trimmed.head()

,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,...,publication_is_software_tool,publication_publisher_name,publication_publisher_prestige,publication_journal_prestige,authorship_total_authors,authorship_vse_author_count,authorship_vse_author_is_primary,authorship_collaboration_scope,summary_abstract_detailed,citations_openalex
0,ARTIFICIAL INTELLIGENCE,ARTIFICIAL INTELLIGENCE,0004-3702,4.6,1.726,Q2,"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1 (D2),"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1,...,False,Elsevier,9.0,9.0,3.0,2.0,True,2.0,While the interpretability of machine learning...,106.0
1,ENVIRONMENT AND BEHAVIOR,ENVIRONMENT AND BEHAVIOR,0013-9165,4.6,1.710,Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),...,False,SAGE Publications,8.0,7.0,3.0,1.0,False,1.0,Several studies have suggested that people mig...,26.0
2,ENVIRONMENT AND BEHAVIOR,ENVIRONMENT AND BEHAVIOR,0013-9165,4.6,1.710,Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),...,False,SAGE Publications,NaN,NaN,3.0,1.0,False,1.0,Several studies have shown that moral licensin...,8.0
3,MANAGEMENT SCIENCE,MANAGEMENT SCIENCE,0025-1909,4.9,4.180,Q1,OPERATIONS RESEARCH & MANAGEMENT SCIENCE - MAN...,Q1 (D1),OPERATIONS RESEARCH & MANAGEMENT SCIENCE - MAN...,Q1 (D2),...,False,Institute for Operations Research and the Mana...,NaN,NaN,3.0,1.0,False,3.0,The authors use a laboratory experiment that i...,64.0
4,REGIONAL STUDIES,REGIONAL STUDIES,0034-3404,4.2,1.343,Q1 (D1),ECONOMICS - GEOGRAPHY,Q1 (D2),ECONOMICS - ENVIRONMENTAL STUDIES - GEOGRAPHY ...,Q1 (D2),...,False,"Routledge Journals, Taylor & Francis Ltd",NaN,NaN,4.0,0.0,False,4.0,Despite being one of the main European Union (...,3.0


In [ ]:
missing_df_clean.head()

,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,...,file_id,file_name,file_type,cc_license,title,journal_title,journal_issn,journal_year,doi,row_id
47,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,1472-8117,7.4,0.929,Q1 (D1),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH - ...,Q1 (D2),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH,Q1 (D1),...,1717,houdek-2023-1-.pdf,published,none,The deep roots of cross-cultural differences i...,The International Journal of Management Education,1472-8117,2023,10.1016/j.ijme.2023.100876,47
48,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,1472-8117,7.4,0.929,Q1 (D1),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH - ...,Q1 (D2),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH,Q1 (D1),...,2004,hasa-houdek-2024-2-.pdf,published,none,Managerial taboos: How the ideal of a manager ...,The International Journal of Management Education,1472-8117,2024,10.1016/j.ijme.2024.100992,48
49,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,1472-8117,7.4,0.929,Q1 (D1),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH - ...,Q1 (D2),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH,Q1 (D1),...,2167,ijme-teaching-with-bc-in-higher-education.pdf,published,none,Teaching with business cases in higher educati...,The International Journal of Management Education,1472-8117,2024,10.1016/j.ijme.2024.101068,49
50,JOURNAL OF CONTEMPORARY EUROPEAN STUDIES,JOURNAL OF CONTEMPORARY EUROPEAN STUDIES,1478-2804,1.8,0.613,Q1 (D1),AREA STUDIES,Q1,AREA STUDIES,Q2,...,1707,do-the-eu-and-ukraine-speak-the-same-language-...,published,none,Do the EU and Ukraine speak the same language?...,Journal of Contemporary European Studies,1478-2804,2023,10.1080/14782804.2022.2163991,50
51,HUMAN RESOURCES FOR HEALTH,HUMAN RESOURCES FOR HEALTH,1478-4491,4.3,1.311,Q1 (D1),HEALTH POLICY & SERVICES - INDUSTRIAL RELATION...,Q1 (D2),HEALTH POLICY & SERVICES - INDUSTRIAL RELATION...,Q1 (D1),...,1080,s12960-021-00630-y.pdf,published,none,Regional inequalities and substitutability of ...,Human Resources for Health,1478-4491,2021,10.1186/s12960-021-00630-y,51


In [ ]:
missing_df_with_features = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/missing_df_features.csv")
missing_df_with_features.head(30)

,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,...,publication_page_length,publication_is_software_tool,publication_publisher_name,publication_publisher_prestige,publication_journal_prestige,authorship_total_authors,authorship_vse_author_count,authorship_vse_author_is_primary,authorship_collaboration_scope,summary_abstract_detailed
0,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,1472-8117,7.4,0.929,Q1 (D1),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH - ...,Q1 (D2),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH,Q1 (D1),...,NaN,False,Elsevier ([colab.ws](https://colab.ws/articles...,NaN,NaN,1.0,1.0,True,1.0,This article presents a thematic review of dee...
1,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,1472-8117,7.4,0.929,Q1 (D1),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH - ...,Q1 (D2),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH,Q1 (D1),...,12.0,False,Elsevier Ltd.,8.0,7.0,2.0,2.0,True,1.0,This conceptual paper explores the seldom-disc...
2,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,INTERNATIONAL JOURNAL OF MANAGEMENT EDUCATION,1472-8117,7.4,0.929,Q1 (D1),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH - ...,Q1 (D2),BUSINESS - EDUCATION & EDUCATIONAL RESEARCH,Q1 (D1),...,15.0,False,Elsevier B.V.,NaN,NaN,6.0,2.0,False,4.0,Although business cases are frequently used in...
3,JOURNAL OF CONTEMPORARY EUROPEAN STUDIES,JOURNAL OF CONTEMPORARY EUROPEAN STUDIES,1478-2804,1.8,0.613,Q1 (D1),AREA STUDIES,Q1,AREA STUDIES,Q2,...,20.0,False,Taylor & Francis,NaN,NaN,3.0,1.0,True,2.0,The concept of resilience has become a leitmot...
4,HUMAN RESOURCES FOR HEALTH,HUMAN RESOURCES FOR HEALTH,1478-4491,4.3,1.311,Q1 (D1),HEALTH POLICY & SERVICES - INDUSTRIAL RELATION...,Q1 (D2),HEALTH POLICY & SERVICES - INDUSTRIAL RELATION...,Q1 (D1),...,NaN,False,Springer Nature (BioMed Central),NaN,NaN,1.0,1.0,True,1.0,Background\n\nAn analysis of the regional dist...
5,CORPORATE SOCIAL RESPONSIBILITY AND ENVIRONMEN...,CORPORATE SOCIAL RESPONSIBILITY AND ENVIRONMEN...,1535-3958,9.1,1.520,Q1 (D1),BUSINESS - ENVIRONMENTAL STUDIES - MANAGEMENT,Q1 (D2),BUSINESS - ENVIRONMENTAL STUDIES - MANAGEMENT,Q1 (D1),...,12.0,False,ERP Environment and John Wiley & Sons Ltd,9.0,8.0,4.0,1.0,True,4.0,This paper investigates how a venture can appl...
6,JOURNAL OF THE EUROPEAN ECONOMIC ASSOCIATION,JOURNAL OF THE EUROPEAN ECONOMIC ASSOCIATION,1542-4766,3.3,5.766,Q1 (D2),ECONOMICS,Q1 (D1),ECONOMICS,Q2,...,33.0,False,Oxford University Press,NaN,NaN,6.0,1.0,False,4.0,This paper provides evidence showing that peop...
7,FINANCE RESEARCH LETTERS,FINANCE RESEARCH LETTERS,1544-6123,6.9,1.076,Q1 (D1),"BUSINESS, FINANCE",Q1 (D2),"BUSINESS, FINANCE",Q1 (D1),...,8.0,False,Elsevier BV,NaN,NaN,2.0,1.0,True,1.0,We propose a novel distribution derived from t...
8,FINANCE RESEARCH LETTERS,FINANCE RESEARCH LETTERS,1544-6123,6.9,1.076,Q1 (D1),"BUSINESS, FINANCE",Q1 (D2),"BUSINESS, FINANCE",Q1 (D1),...,5.0,False,Elsevier Inc.,NaN,NaN,2.0,2.0,True,1.0,"Dishonest actions, while beneficial to perpetr..."
9,FINANCE RESEARCH LETTERS,FINANCE RESEARCH LETTERS,1544-6123,6.9,1.076,Q1 (D1),"BUSINESS, FINANCE",Q1 (D2),"BUSINESS, FINANCE",Q1 (D1),...,NaN,False,Elsevier,NaN,NaN,4.0,1.0,False,NaN,We employ alpha-stable distribution to dynamic...


In [ ]:
combined_df_trimmed.shape

(47, 37)

In [ ]:
missing_df_with_features.shape

(23, 37)

In [ ]:
missing_df_with_features["citations_openalex"] = missing_df_with_features["doi"].apply(get_openalex_citations)

In [ ]:
combined_df_fixed = pd.concat(
    [combined_df_trimmed, missing_df_with_features],
    ignore_index=True
).sort_values("row_id").reset_index(drop=True)

In [ ]:
combined_df_fixed.to_csv(
    "/content/drive/MyDrive/Colab Notebooks/df_LLM_features_fixed.csv",
    index=False
)

In [ ]:
# nový prompt

In [ ]:
Your task is to perform a comprehensive evaluation of a research article. You will be provided with an article abstract and, potentially, other metadata such as author affiliations, journal information, or citation counts. Note that this metadata may not always be supplied. Your analysis must result in a single, valid JSON object.

The JSON object must contain two top-level keys: `structured_evaluation` and `narrative_review`.

---
### PART 1: `structured_evaluation` Object
---
This object will contain a detailed breakdown of the article based on specific criteria. For each criterion listed below, provide an object containing its `value` and a one- or two-sentence `justification`.

**Criteria Details:**

1.  `incrementalScore`: Assesses if the work is an incremental extension (lower score) or a breakthrough (higher score).
    -   Type: Integer, Range: [1, 5], Best: 5
2.  `isSoftwareTool`: True if the publication's main goal is to announce software, a database, or a toolbox.
    -   Type: Boolean
3.  `isRegionallyFocused`: True if the study has a narrow geographical, historical, or thematic focus that limits its international relevance.
    -   Type: Boolean, Best: False
4.  `methodologicalRigorScore`: Evaluates the soundness and appropriateness of the research methods and experimental design.
    -   Type: Integer, Range: [1, 5], Best: 5
5.  `analyticalDataRigorScore`: Evaluates the depth, quality, and robustness of the data analysis and statistical methods.
    -   Type: Integer, Range: [1, 5], Best: 5
6.  `practicalApplicabilityScore`: Assesses the potential to apply the research in real-world scenarios (clinical, industrial, etc.).
    -   Type: Integer, Range: [1, 5], Best: 5
7.  `societalPolicyRelevanceScore`: Assesses the research's relevance to public health, policy, or other societal issues.
    -   Type: Integer, Range: [1, 5], Best: 5
8.  `futureResearchPotential`: Assesses the extent to which the paper opens new avenues for future studies.
    -   Type: Integer, Range: [1, 5], Best: 5
9.  `vseAuthorIsPrimary`: True if an author from VŠE (Prague University of Economics and Business) is a first or corresponding author.
    -   Type: Boolean, Best: True
10. `collaborationScope`: The number of unique countries in author affiliations, indicating international collaboration.
    -   Type: Integer, Range: [1, ∞)
11. `totalAuthors`: The total number of authors on the publication.
    -   Type: Integer, Range: [1, ∞)
12. `vseAuthorCount`: The number of authors affiliated with the Prague University of Economics and Business.
    -   Type: Integer, Range: [0, ∞)
13. `journalPrestige`: A score of the journal's perceived reputation within its field.
    -   Type: Integer, Range: [0, 10], Best: 10
14. `publisherPrestige`: A score of the publisher's perceived prestige in the academic community.
    -   Type: Integer, Range: [0, 10], Best: 10
15. `articleType`: The type of article.
    -   Type: String, Options: ["research_paper", "review", "editorial", "brief_report", "letter", "applications_note", etc.]
16. `pageLength`: The total number of pages in the article.
    -   Type: Integer, Range: [1, ∞)

---
### PART 2: `narrative_review` Object
---
This object will contain a formal, evidence-based assessment of the research article, following the Czech M17+ evaluation style. It must have two fields: `evaluation_text` and `proposed_grade`.

**Instructions for `evaluation_text`:**
Generate a single, cohesive paragraph in a formal, objective, and direct tone. The review must be evidence-centric, grounding its assessment in the research's methodology, significance, and contribution to its field. Follow this structure:
1.  **Initial Summary & Context:** Briefly state the research topic and contextualize the quality of the publication venue (if known).
2.  **Analysis of Scientific Merit:** Assess the work's **originality**, **significance**, and **difficulty**.
3.  **Supporting Evidence & Critique:** Use available metrics (e.g., citation count) to support the assessment. Critically evaluate any limitations.
4.  **Concluding Synthesis:** Summarize the assessment, leading to the final proposed grade.

**Key Factors for Critical Assessment (based on M17+ analysis):**
When formulating your critique, consider these common reasons why articles with high-prestige indicators (like a high AIS) might not receive the top grade:

* **Lack of Breakthrough Innovation:** Is the work a solid, high-quality contribution but ultimately incremental rather than a true breakthrough that changes the field? Does it primarily extend previous results?
* **Nature of the Publication:** Is the article a review, meta-analysis, or survey? While valuable for summarizing knowledge, these often lack the originality of primary research. Is it a non-standard format like a "Brief Report," "Letter to the Editor," or "Applications Note" which limits its scope?
* **Methodological & Validation Limits:** Does the study use an overly simple methodology or basic descriptive statistics where more sophisticated analysis is warranted? Are there weaknesses in the sample (e.g., small, non-representative)? Is the validation insufficient, performed only on simple examples, or lacking comparison with state-of-the-art alternatives?
* **Limited Scope & Generalizability:** Is the study's focus on a specific geographic region or historical context so narrow that its findings are not easily transferable? Are there concerns about the scalability or practical applicability of the proposed methods?
* **Low Demonstrated Impact:** If citation data is available, is the citation count low for its field and age? Is there a high proportion of self-citations? For software, is there low community engagement?
* **Character of the Contribution:** Is the work primarily an engineering or implementation effort (e.g., a software toolbox for existing methods) rather than a fundamental contribution to knowledge?
* **Author Contribution Context:** (If discernible from metadata) Is the author's role marginal in a large consortium or a commercially sponsored study? Is there a lack of international collaboration?

**Instructions for `proposed_grade`:**
Provide a final grade as an integer from 1 (best) to 5 (worst), based on your synthesis. This grade reflects the M17+ evaluation scale:
-   **1: Špičkový (World-leading)** - Groundbreaking work that significantly advances the field. Reserved for true breakthroughs, not just excellent publications.
-   **2: Vynikající (Excellent)** - A significant contribution at an excellent international level, but may not be a complete paradigm shift. Often the grade for high-quality but incremental work.
-   **3: Mezinárodně uznávaný (Internationally recognized) / Velmi dobrý (Very good)** - Solid, competent research recognized internationally, but may represent a more partial or specialized contribution.
-   **4: Průměrný (Average) / Národně uznatelný (Nationally recognized)** - A result of average quality with primarily local or national significance.
-   **5: Podprůměrný (Below average)** - The result has significant flaws, is a non-scientific format (e.g., letter to editor), or does not meet the standard of nationally recognized research.

---
### FINAL INSTRUCTION
---
Generate only the single, valid JSON object described above. Do not include any text or formatting outside of the JSON structure itself.


In [ ]:
#starý prompt - udělat procházení a mergování podle ID - pro starý prompt abstrakt, nový dát full text

In [ ]:
# nový prompt - udělat nové sloupce tam dát celkové hodnocení + dílčí hodnocení

In [ ]:
import json

def generate_categories_for_abstract_ChatGPT(abstract):
    response = client.chat.completions.create(
        model="gpt-5-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    "Your job is to assign scores to an abstact of research paper in the following evaluation categories: "
                    "\"clarity\", \"field_contribution\", \"methodological_appropriateness\", \"evidence_quality\", "
                    "\"citation_quality\", \"originality_of_methods_or_approach\", \"readability\", \"rigor\", "
                    "\"novelty\", \"replicability\", \"accessibility\", \"grammar\".\n\n"

                    "Each category must be rated on a 7-point ordinal scale, with the following meaning:\n"
                    "\"1\" = \"Very low level – severe shortcomings, does not meet academic standards.\"\n"
                    "\"2\" = \"Low level – major weaknesses, limited quality.\"\n"
                    "\"3\" = \"Below average – partially acceptable, but with significant limitations.\"\n"
                    "\"4\" = \"Average – meets basic academic standards, nothing exceptional.\"\n"
                    "\"5\" = \"Above average – generally good, only minor shortcomings.\"\n"
                    "\"6\" = \"High level – solid and reliable, only very small weaknesses.\"\n"
                    "\"7\" = \"Exceptional – outstanding, meets the highest standards with no relevant flaws.\"\n\n"

                    "After rating all categories, you must also provide a predicted overall score of the paper "
                    "according to the Czech M17+ methodology for research assessment. "
                    "Choose one value from \"1\", \"2\", \"3\", \"4\", \"5\". \n"
                    "\"1\" = \"Špičkový (World-leading)\"\n"
                    "\"2\" = \"Vynikající (Excellent)\"\n"
                    "\"3\" = \"Mezinárodně uznávaný (Internationally recognized) / Velmi dobrý (Very good)\"\n"
                    "\"4\" = \"Průměrný (Average) / Národně uznatelný (Nationally recognized)\"\n"
                    "\"5\" = \"Podprůměrný (Below average)\"\n"


                    "Base your judgment only on the provided TARGET paper’s abstract. "
                    "Be consistent: always choose exactly one value (\"1\"–\"7\") for each category, and exactly one value (\"1\"–\"5\") for the overall \"m17_plus_score\". "
                    "Do not invent new categories or values.\n\n"

                    "Return strict JSON only, exactly like:\n"
                    "{\n"
                    "  \"clarity\": \"value\",\n"
                    "  \"field_contribution\": \"value\",\n"
                    "  \"methodological_appropriateness\": \"value\",\n"
                    "  \"evidence_quality\": \"value\",\n"
                    "  \"citation_quality\": \"value\",\n"
                    "  \"originality_of_methods_or_approach\": \"value\",\n"
                    "  \"readability\": \"value\",\n"
                    "  \"rigor\": \"value\",\n"
                    "  \"novelty\": \"value\",\n"
                    "  \"replicability\": \"value\",\n"
                    "  \"accessibility\": \"value\",\n"
                    "  \"grammar\": \"value\",\n"
                    "  \"m17_plus_score\": \"value\"\n"
                    "}"
                ),
            },
            {
                "role": "user",
                "content": f"Abstract of paper to be evaluated:\n\n{abstract}",
            },
        ],
    )
    output = response.choices[0].message.content
    try:
        data = json.loads(output)
    except json.JSONDecodeError:
        data = {}
    return data


In [ ]:
#načtení datasetu
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/df_LLM_features_fixed_abstracts.csv",
                 sep=',',
                 encoding="latin1")

In [ ]:
merged_df.shape

(67, 40)

In [ ]:
# načtení datasetu
dataset_11 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset.csv", sep=";")

In [ ]:
dataset_11.head(12)

,nazev,issn,doi,abstrakt,pdf,id
0,The Status Quo and Belief Polarization of Inat...,1945-7669,10.1257/mic.20210352,We show that rational but inattentive agents c...,ano,1
1,Kahneman in Quotes and Reflections,1047-840X,10.1080/1047840X.2024.2366813,In this retrospective honoring the exemplary p...,ano,2
2,A review of possible effects of cognitive bias...,0004-3702,10.1016/j.artint.2021.103458,While the interpretability of machine learning...,ano,3
3,"Bank capital, lending, and regulation: A meta-...",0950-0804,10.1111/joes.12560,We collected over 1600 estimates on the relati...,ne,4
4,Financial crime and punishment: A meta-analysis,0950-0804,10.1111/joes.12580,We examine how the publication of intentional ...,ano,5
5,Sparse solution of least-squares twin multi-cl...,0893-6080,10.1016/j.neunet.2023.07.039,"In the realm of multi-class classification, th...",ne,6
6,The Co-Ontological Securities of Gated Lifewor...,1749-5679,10.1093/ips/olae028,This article returns to the existentialist roo...,ano,7
7,Restaurant Hygiene Attributes and Consumers' F...,1873-1384,10.1016/j.jretconser.2022.102972,Restaurant unhygienic affairs have concerned c...,ano,8
8,Does the level of e-government affect value-ad...,1944-2866,10.1002/poi3.389,The digitalization of the economy and state ad...,ne,9
9,Rainfall variability and labor allocation in U...,1060-586X,10.1080/1060586X.2024.2309822,Employing a novel georeferenced household surv...,ano,10


In [ ]:
# Ukládáme jen celkové hodnocení m17_plus_score
results = {"id": [], "abstrakt": [], "old_prompt": []}

for row in dataset_11.itertuples(index=False):
    abstract = getattr(row, "abstrakt").strip()
    row_id = getattr(row, "id")

    scores = generate_categories_for_abstract_ChatGPT(abstract)

    results["id"].append(row_id)
    results["abstrakt"].append(abstract)
    results["old_prompt"].append(scores.get("m17_plus_score", None))

results_df = pd.DataFrame(results)

# merge podle 'row_id'
merged_df = dataset_11.merge(results_df, on="id", how="left")

# uložíme CSV
merged_df.to_csv("/content/drive/MyDrive/Colab Notebooks/dataset_11_old_prompt.csv", sep=",", index=False, encoding="utf-8-sig")
merged_df.head(12)


,nazev,issn,doi,abstrakt_x,pdf,id,abstrakt_y,old_prompt
0,The Status Quo and Belief Polarization of Inat...,1945-7669,10.1257/mic.20210352,We show that rational but inattentive agents c...,ano,1,We show that rational but inattentive agents c...,3
1,Kahneman in Quotes and Reflections,1047-840X,10.1080/1047840X.2024.2366813,In this retrospective honoring the exemplary p...,ano,2,In this retrospective honoring the exemplary p...,4
2,A review of possible effects of cognitive bias...,0004-3702,10.1016/j.artint.2021.103458,While the interpretability of machine learning...,ano,3,While the interpretability of machine learning...,4
3,"Bank capital, lending, and regulation: A meta-...",0950-0804,10.1111/joes.12560,We collected over 1600 estimates on the relati...,ne,4,We collected over 1600 estimates on the relati...,3
4,Financial crime and punishment: A meta-analysis,0950-0804,10.1111/joes.12580,We examine how the publication of intentional ...,ano,5,We examine how the publication of intentional ...,3
5,Sparse solution of least-squares twin multi-cl...,0893-6080,10.1016/j.neunet.2023.07.039,"In the realm of multi-class classification, th...",ne,6,"In the realm of multi-class classification, th...",3
6,The Co-Ontological Securities of Gated Lifewor...,1749-5679,10.1093/ips/olae028,This article returns to the existentialist roo...,ano,7,This article returns to the existentialist roo...,3
7,Restaurant Hygiene Attributes and Consumers' F...,1873-1384,10.1016/j.jretconser.2022.102972,Restaurant unhygienic affairs have concerned c...,ano,8,Restaurant unhygienic affairs have concerned c...,4
8,Does the level of e-government affect value-ad...,1944-2866,10.1002/poi3.389,The digitalization of the economy and state ad...,ne,9,The digitalization of the economy and state ad...,3
9,Rainfall variability and labor allocation in U...,1060-586X,10.1080/1060586X.2024.2309822,Employing a novel georeferenced household surv...,ano,10,Employing a novel georeferenced household surv...,3


In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 6.5 MB/s eta 0:00:00


In [ ]:
# --- imports
import os, json
import pandas as pd

# PDF reader
try:
    from pypdf import PdfReader
except Exception:
    from PyPDF2 import PdfReader  # fallback


def read_pdf_text(pdf_path, max_chars=50000):
    """Načte text z PDF. Pro jistotu ořízne na max_chars (kvůli limitu kontextu)."""
    if not pdf_path or not os.path.exists(pdf_path):
        return ""
    try:
        reader = PdfReader(pdf_path)
        parts = []
        for page in reader.pages:
            try:
                parts.append(page.extract_text() or "")
            except Exception:
                continue
        text = "\n".join(parts).strip()
        if max_chars and len(text) > max_chars:
            text = text[:max_chars]
        return text
    except Exception:
        return ""


def generate_evaluation_for_fulltext_ChatGPT(full_text, metadata=None):
    """
    Pošle modelu full-text (případně metadata) a vrátí striktní JSON.
    Očekává klíče: structured_evaluation a narrative_review.
    """

    if not full_text:
        return {}

    system_prompt = (
        "Your task is to perform a comprehensive evaluation of a research article. "
        "You will be provided with the FULL TEXT of an article and, potentially, other metadata "
        "such as author affiliations, journal information, or citation counts. "
        "Note that this metadata may not always be supplied. Your analysis must result in a single, valid JSON object.\n\n"
        "The JSON object must contain two top-level keys: `structured_evaluation` and `narrative_review`.\n"
        "---\n"
        "### PART 1: `structured_evaluation` Object\n"
        "---\n"
        "This object will contain a detailed breakdown of the article based on specific criteria. "
        "For each criterion listed below, provide an object containing its `value` and a one- or two-sentence `justification`.\n\n"
        "**Criteria Details:**\n\n"
        "1.  `incrementalScore`: Assesses if the work is an incremental extension (lower score) or a breakthrough (higher score).\n"
        "    -   Type: Integer, Range: [1, 5], Best: 5\n"
        "2.  `isSoftwareTool`: True if the publication's main goal is to announce software, a database, or a toolbox.\n"
        "    -   Type: Boolean\n"
        "3.  `isRegionallyFocused`: True if the study has a narrow geographical, historical, or thematic focus that limits its international relevance.\n"
        "    -   Type: Boolean, Best: False\n"
        "4.  `methodologicalRigorScore`: Evaluates the soundness and appropriateness of the research methods and experimental design.\n"
        "    -   Type: Integer, Range: [1, 5], Best: 5\n"
        "5.  `analyticalDataRigorScore`: Evaluates the depth, quality, and robustness of the data analysis and statistical methods.\n"
        "    -   Type: Integer, Range: [1, 5], Best: 5\n"
        "6.  `practicalApplicabilityScore`: Assesses the potential to apply the research in real-world scenarios (clinical, industrial, etc.).\n"
        "    -   Type: Integer, Range: [1, 5], Best: 5\n"
        "7.  `societalPolicyRelevanceScore`: Assesses the research's relevance to public health, policy, or other societal issues.\n"
        "    -   Type: Integer, Range: [1, 5], Best: 5\n"
        "8.  `futureResearchPotential`: Assesses the extent to which the paper opens new avenues for future studies.\n"
        "    -   Type: Integer, Range: [1, 5], Best: 5\n"
        "9.  `vseAuthorIsPrimary`: True if an author from VŠE (Prague University of Economics and Business) is a first or corresponding author.\n"
        "    -   Type: Boolean, Best: True\n"
        "10. `collaborationScope`: The number of unique countries in author affiliations, indicating international collaboration.\n"
        "    -   Type: Integer, Range: [1, ∞)\n"
        "11. `totalAuthors`: The total number of authors on the publication.\n"
        "    -   Type: Integer, Range: [1, ∞)\n"
        "12. `vseAuthorCount`: The number of authors affiliated with the Prague University of Economics and Business.\n"
        "    -   Type: Integer, Range: [0, ∞)\n"
        "13. `journalPrestige`: A score of the journal's perceived reputation within its field.\n"
        "    -   Type: Integer, Range: [0, 10], Best: 10\n"
        "14. `publisherPrestige`: A score of the publisher's perceived prestige in the academic community.\n"
        "    -   Type: Integer, Range: [0, 10], Best: 10\n"
        "15. `articleType`: The type of article.\n"
        "    -   Type: String, Options: [\"research_paper\", \"review\", \"editorial\", \"brief_report\", \"letter\", \"applications_note\", etc.]\n"
        "16. `pageLength`: The total number of pages in the article.\n"
        "    -   Type: Integer, Range: [1, ∞)\n\n"
        "---\n"
        "### PART 2: `narrative_review` Object\n"
        "---\n"
        "This object will contain a formal, evidence-based assessment of the research article, following the Czech M17+ evaluation style. "
        "It must have two fields: `evaluation_text` and `proposed_grade`.\n\n"
        "**Instructions for `evaluation_text`:**\n"
        "Generate a single, cohesive paragraph in a formal, objective, and direct tone. The review must be evidence-centric, grounding its assessment in the research's methodology, significance, and contribution to its field. Follow this structure:\n"
        "1.  Initial Summary & Context.\n"
        "2.  Analysis of Scientific Merit (originality, significance, difficulty).\n"
        "3.  Supporting Evidence & Critique (use metrics if available; discuss limits).\n"
        "4.  Concluding Synthesis leading to the final proposed grade.\n\n"
        "Key factors for critical assessment (M17+): lack of breakthrough innovation; nature of publication (reviews/brief reports, etc.); "
        "methodological/validation limits; limited scope/generalizability; low demonstrated impact; primarily engineering/implementation character; "
        "author contribution context.\n\n"
        "**Instructions for `proposed_grade`:** Provide an integer 1–5 per the M17+ scale described.\n\n"
        "### FINAL INSTRUCTION\n"
        "Generate only the single, valid JSON object described above. Do not include any text or formatting outside of the JSON structure itself."
    )

    user_payload = {
        "full_text": full_text,
        "two_year_mean_citedness": metadata.get("two_year_mean_citedness", None),
        "metadata": metadata or {}
    }

    response = client.chat.completions.create(
        model="gpt-5-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": json.dumps(user_payload)}
        ],

    )

    output = response.choices[0].message.content
    try:
        data = json.loads(output)
    except json.JSONDecodeError:
        data = {}
    return data



PDF_DIR = "/content/drive/MyDrive/Colab Notebooks/pdfs"

rows_out = []
for row in df_final.itertuples(index=False):
    row_id = getattr(row, "id")
    abstract = getattr(row, "abstrakt", None)
    pdf_stub = getattr(row, "pdf_file_name", None)
    pdf_path = os.path.join(PDF_DIR, f"{pdf_stub}.pdf") if pdf_stub else None


    metadata = {
        "title": getattr(row, "nazev", None),
        "issn": getattr(row, "issn", None),
        "doi": getattr(row, "doi", None),
        "two_year_mean_citedness": getattr(row, "2yr_mean_citedness", None),
        "had_pdf_flag": getattr(row, "pdf", None),
        "abstract": abstract
    }

    full_text = read_pdf_text(pdf_path)

    result = generate_evaluation_for_fulltext_ChatGPT(full_text, metadata=metadata)


    flat = {"id": row_id, "pdf_path": pdf_path}

    # structured_evaluation -> rozbalit do sloupců `<key>_value` a `<key>_justification`
    se = result.get("structured_evaluation", {}) or {}
    for k, v in se.items():
        if isinstance(v, dict):
            flat[f"{k}_value"] = v.get("value")
            flat[f"{k}_justification"] = v.get("justification")
        else:

            flat[f"{k}_value"] = v
            flat[f"{k}_justification"] = None


    nr = result.get("narrative_review", {}) or {}
    flat["narrative_evaluation_text"] = nr.get("evaluation_text")
    flat["narrative_proposed_grade"] = nr.get("proposed_grade")


    flat["raw_json"] = json.dumps(result, ensure_ascii=False)

    rows_out.append(flat)

# do DataFrame
results_df = pd.DataFrame(rows_out)

# merge zpět na původní dataset podle 'id'
merged_df = df_final.merge(results_df, on="id", how="left")


out_path = "/content/drive/MyDrive/Colab Notebooks/dataset_3_missing_new_prompt.csv"
merged_df.to_csv(out_path, sep=",", index=False, encoding="utf-8-sig")


merged_df.head(12)


,nazev,issn,doi,abstrakt_x,pdf,id,abstrakt_y,old_prompt,2yr_mean_citedness,pdf_file_name,...,journalPrestige_justification,publisherPrestige_value,publisherPrestige_justification,articleType_value,articleType_justification,pageLength_value,pageLength_justification,narrative_evaluation_text,narrative_proposed_grade,raw_json
0,"Bank capital, lending, and regulation: A meta-...",0950-0804,10.1111/joes.12560,We collected over 1600 estimates on the relati...,ne,4,We collected over 1600 estimates on the relati...,3,6.250000,Mal,...,"Journal of Economic Surveys is a respected, hi...",8,"The publisher, John Wiley & Sons, is a major a...",research_paper,This is an original empirical research article...,29,"The journal citation indicates pages 823–851, ...",This article presents a comprehensive meta-ana...,2,"{""structured_evaluation"": {""incrementalScore"":..."
1,Sparse solution of least-squares twin multi-cl...,0893-6080,10.1016/j.neunet.2023.07.039,"In the realm of multi-class classification, th...",ne,6,"In the realm of multi-class classification, th...",3,6.558097,Hlad,...,Neural Networks (Elsevier) is a well-establish...,8,Elsevier is a major commercial academic publis...,research_paper,The manuscript reports original methodological...,16,The article appears in Neural Networks across ...,"This manuscript proposes PLSTKSVC, an lp-norm ...",3,"{""structured_evaluation"": {""incrementalScore"":..."
2,Does the level of e-government affect value-ad...,1944-2866,10.1002/poi3.389,The digitalization of the economy and state ad...,ne,9,The digitalization of the economy and state ad...,3,3.971963,Zid,...,Policy & Internet is a recognized interdiscipl...,8,"The article is published by Wiley, a major int...",research_paper,The manuscript reports original empirical rese...,21,The journal pagination in the provided text sp...,This empirical paper examines whether higher l...,2,"{""structured_evaluation"": {""incrementalScore"":..."


In [ ]:
test_df = dataset_11.head(1).copy()
test_df.head()

,nazev,issn,doi,abstrakt,pdf,id,2yr_mean_citedness,pdf_file_name
0,The Status Quo and Belief Polarization of Inat...,1945-7669,10.1257/mic.20210352,We show that rational but inattentive agents c...,ano,1,2.897436,The_status


In [ ]:
merged_df[['id', 'old_prompt', 'narrative_proposed_grade']].head(20)

In [ ]:
# načtení df se známkou ze starého promptu
df_final = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset_11_old_prompt.csv", sep=",")
df_final.head(
    12
)

,nazev,issn,doi,abstrakt_x,pdf,id,abstrakt_y,old_prompt
0,The Status Quo and Belief Polarization of Inat...,1945-7669,10.1257/mic.20210352,We show that rational but inattentive agents c...,ano,1,We show that rational but inattentive agents c...,3
1,Kahneman in Quotes and Reflections,1047-840X,10.1080/1047840X.2024.2366813,In this retrospective honoring the exemplary p...,ano,2,In this retrospective honoring the exemplary p...,4
2,A review of possible effects of cognitive bias...,0004-3702,10.1016/j.artint.2021.103458,While the interpretability of machine learning...,ano,3,While the interpretability of machine learning...,4
3,"Bank capital, lending, and regulation: A meta-...",0950-0804,10.1111/joes.12560,We collected over 1600 estimates on the relati...,ne,4,We collected over 1600 estimates on the relati...,3
4,Financial crime and punishment: A meta-analysis,0950-0804,10.1111/joes.12580,We examine how the publication of intentional ...,ano,5,We examine how the publication of intentional ...,3
5,Sparse solution of least-squares twin multi-cl...,0893-6080,10.1016/j.neunet.2023.07.039,"In the realm of multi-class classification, th...",ne,6,"In the realm of multi-class classification, th...",3
6,The Co-Ontological Securities of Gated Lifewor...,1749-5679,10.1093/ips/olae028,This article returns to the existentialist roo...,ano,7,This article returns to the existentialist roo...,3
7,Restaurant Hygiene Attributes and Consumers' F...,1873-1384,10.1016/j.jretconser.2022.102972,Restaurant unhygienic affairs have concerned c...,ano,8,Restaurant unhygienic affairs have concerned c...,4
8,Does the level of e-government affect value-ad...,1944-2866,10.1002/poi3.389,The digitalization of the economy and state ad...,ne,9,The digitalization of the economy and state ad...,3
9,Rainfall variability and labor allocation in U...,1060-586X,10.1080/1060586X.2024.2309822,Employing a novel georeferenced household surv...,ano,10,Employing a novel georeferenced household surv...,3


In [ ]:
# získání IF
import requests

def get_if_from_openalex(issn):
    url = f"https://api.openalex.org/sources?filter=issn:{issn}"
    response = requests.get(url)

    if response.status_code != 200:
        return None  # vrátíme jen číslo nebo None

    data = response.json()

    # Když se ISSN nenajde v OpenAlex
    if not data["results"]:
        return None

    journal = data["results"][0]
    if_like = journal.get("summary_stats", {}).get("2yr_mean_citedness", None)

    return if_like

result = []

for row in df_final.itertuples(index=False):
    issn = getattr(row, "issn")
    impactF = get_if_from_openalex(issn)
    result.append(impactF)

df_final['IF'] = result
df_final.head(12)


,nazev,issn,doi,abstrakt_x,pdf,id,abstrakt_y,old_prompt,IF
0,The Status Quo and Belief Polarization of Inat...,1945-7669,10.1257/mic.20210352,We show that rational but inattentive agents c...,ano,1,We show that rational but inattentive agents c...,3,2.897436
1,Kahneman in Quotes and Reflections,1047-840X,10.1080/1047840X.2024.2366813,In this retrospective honoring the exemplary p...,ano,2,In this retrospective honoring the exemplary p...,4,1.179487
2,A review of possible effects of cognitive bias...,0004-3702,10.1016/j.artint.2021.103458,While the interpretability of machine learning...,ano,3,While the interpretability of machine learning...,4,2.916898
3,"Bank capital, lending, and regulation: A meta-...",0950-0804,10.1111/joes.12560,We collected over 1600 estimates on the relati...,ne,4,We collected over 1600 estimates on the relati...,3,6.250000
4,Financial crime and punishment: A meta-analysis,0950-0804,10.1111/joes.12580,We examine how the publication of intentional ...,ano,5,We examine how the publication of intentional ...,3,NaN
5,Sparse solution of least-squares twin multi-cl...,0893-6080,10.1016/j.neunet.2023.07.039,"In the realm of multi-class classification, th...",ne,6,"In the realm of multi-class classification, th...",3,6.558097
6,The Co-Ontological Securities of Gated Lifewor...,1749-5679,10.1093/ips/olae028,This article returns to the existentialist roo...,ano,7,This article returns to the existentialist roo...,3,2.957447
7,Restaurant Hygiene Attributes and Consumers' F...,1873-1384,10.1016/j.jretconser.2022.102972,Restaurant unhygienic affairs have concerned c...,ano,8,Restaurant unhygienic affairs have concerned c...,4,15.694524
8,Does the level of e-government affect value-ad...,1944-2866,10.1002/poi3.389,The digitalization of the economy and state ad...,ne,9,The digitalization of the economy and state ad...,3,3.971963
9,Rainfall variability and labor allocation in U...,1060-586X,10.1080/1060586X.2024.2309822,Employing a novel georeferenced household surv...,ano,10,Employing a novel georeferenced household surv...,3,3.350000


In [ ]:
x = ['The_status', 'Kahneman', 'A_review', None, 'Financial_crime', None,
     'CoOntological', 'Restaurant', None, 'Rainfall', 'Leverage']

df_final['pdf_file_name'] = x
df_final.head(12)


,nazev,issn,doi,abstrakt_x,pdf,id,abstrakt_y,old_prompt,IF,pdf_file_name
0,The Status Quo and Belief Polarization of Inat...,1945-7669,10.1257/mic.20210352,We show that rational but inattentive agents c...,ano,1,We show that rational but inattentive agents c...,3,2.897436,The_status
1,Kahneman in Quotes and Reflections,1047-840X,10.1080/1047840X.2024.2366813,In this retrospective honoring the exemplary p...,ano,2,In this retrospective honoring the exemplary p...,4,1.179487,Kahneman
2,A review of possible effects of cognitive bias...,0004-3702,10.1016/j.artint.2021.103458,While the interpretability of machine learning...,ano,3,While the interpretability of machine learning...,4,2.916898,A_review
3,"Bank capital, lending, and regulation: A meta-...",0950-0804,10.1111/joes.12560,We collected over 1600 estimates on the relati...,ne,4,We collected over 1600 estimates on the relati...,3,6.250000,None
4,Financial crime and punishment: A meta-analysis,0950-0804,10.1111/joes.12580,We examine how the publication of intentional ...,ano,5,We examine how the publication of intentional ...,3,NaN,Financial_crime
5,Sparse solution of least-squares twin multi-cl...,0893-6080,10.1016/j.neunet.2023.07.039,"In the realm of multi-class classification, th...",ne,6,"In the realm of multi-class classification, th...",3,6.558097,None
6,The Co-Ontological Securities of Gated Lifewor...,1749-5679,10.1093/ips/olae028,This article returns to the existentialist roo...,ano,7,This article returns to the existentialist roo...,3,2.957447,CoOntological
7,Restaurant Hygiene Attributes and Consumers' F...,1873-1384,10.1016/j.jretconser.2022.102972,Restaurant unhygienic affairs have concerned c...,ano,8,Restaurant unhygienic affairs have concerned c...,4,15.694524,Restaurant
8,Does the level of e-government affect value-ad...,1944-2866,10.1002/poi3.389,The digitalization of the economy and state ad...,ne,9,The digitalization of the economy and state ad...,3,3.971963,None
9,Rainfall variability and labor allocation in U...,1060-586X,10.1080/1060586X.2024.2309822,Employing a novel georeferenced household surv...,ano,10,Employing a novel georeferenced household surv...,3,3.350000,Rainfall


In [ ]:
df_final = df_final[df_final['pdf'] == 'ne']

In [ ]:
df_final = df_final.rename(columns={'IF': '2yr_mean_citedness'})

In [ ]:
df_final['pdf_file_name'] = ['Mal', 'Hlad', 'Zid']

In [ ]:
df_final.head(12)

,nazev,issn,doi,abstrakt_x,pdf,id,abstrakt_y,old_prompt,2yr_mean_citedness,pdf_file_name
3,"Bank capital, lending, and regulation: A meta-...",0950-0804,10.1111/joes.12560,We collected over 1600 estimates on the relati...,ne,4,We collected over 1600 estimates on the relati...,3,6.250000,Mal
5,Sparse solution of least-squares twin multi-cl...,0893-6080,10.1016/j.neunet.2023.07.039,"In the realm of multi-class classification, th...",ne,6,"In the realm of multi-class classification, th...",3,6.558097,Hlad
8,Does the level of e-government affect value-ad...,1944-2866,10.1002/poi3.389,The digitalization of the economy and state ad...,ne,9,The digitalization of the economy and state ad...,3,3.971963,Zid


In [ ]:
df_final.to_csv("/content/drive/MyDrive/Colab Notebooks/df_for_new_prompt.csv", sep=",", index=False, encoding="utf-8-sig")

In [ ]:
merged_df[['id', 'old_prompt', 'narrative_proposed_grade']].head(20)

,id,old_prompt,narrative_proposed_grade
0,1,3,2
1,2,4,3
2,3,4,4
3,5,3,2
4,7,3,3
5,8,4,3
6,10,3,3
7,11,4,3


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/df_LLM_features_fixed_abstracts.csv"
df_70 = pd.read_csv(path, encoding="latin-1", sep=None, engine="python")
df_70.head()

,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,...,publication_is_software_tool,publication_publisher_name,publication_publisher_prestige,publication_journal_prestige,authorship_total_authors,authorship_vse_author_count,authorship_vse_author_is_primary,authorship_collaboration_scope,summary_abstract_detailed,citations_openalex
0,ARTIFICIAL INTELLIGENCE,ARTIFICIAL INTELLIGENCE,0004-3702,4.6,1.726,Q2,"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1 (D2),"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1,...,False,Elsevier,9.0,9.0,3.0,2.0,True,2.0,While the interpretability of machine learning...,106.0
1,ENVIRONMENT AND BEHAVIOR,ENVIRONMENT AND BEHAVIOR,0013-9165,4.6,1.710,Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),...,False,SAGE Publications,8.0,7.0,3.0,1.0,False,1.0,Several studies have suggested that people mig...,26.0
2,ENVIRONMENT AND BEHAVIOR,ENVIRONMENT AND BEHAVIOR,0013-9165,4.6,1.710,Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),...,False,SAGE Publications,NaN,NaN,3.0,1.0,False,1.0,Several studies have shown that moral licensin...,8.0
3,MANAGEMENT SCIENCE,MANAGEMENT SCIENCE,0025-1909,4.9,4.180,Q1,OPERATIONS RESEARCH & MANAGEMENT SCIENCE - MAN...,Q1 (D1),OPERATIONS RESEARCH & MANAGEMENT SCIENCE - MAN...,Q1 (D2),...,False,Institute for Operations Research and the Mana...,NaN,NaN,3.0,1.0,False,3.0,The authors use a laboratory experiment that i...,64.0
4,REGIONAL STUDIES,REGIONAL STUDIES,0034-3404,4.2,1.343,Q1 (D1),ECONOMICS - GEOGRAPHY,Q1 (D2),ECONOMICS - ENVIRONMENTAL STUDIES - GEOGRAPHY ...,Q1 (D2),...,False,"Routledge Journals, Taylor & Francis Ltd",NaN,NaN,4.0,0.0,False,4.0,Despite being one of the main European Union (...,3.0


In [ ]:
df_70["id"] = range(1, len(df_70) + 1)

In [ ]:
df_70_test = df_70.head(1).copy()
df_70_test.head()

,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,...,publication_publisher_name,publication_publisher_prestige,publication_journal_prestige,authorship_total_authors,authorship_vse_author_count,authorship_vse_author_is_primary,authorship_collaboration_scope,summary_abstract_detailed,citations_openalex,id
0,ARTIFICIAL INTELLIGENCE,ARTIFICIAL INTELLIGENCE,0004-3702,4.6,1.726,Q2,"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1 (D2),"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1,...,Elsevier,9.0,9.0,3.0,2.0,True,2.0,While the interpretability of machine learning...,106.0,1


In [ ]:
# --- imports
import os, json
import pandas as pd


os.environ["AZURE_OPENAI_API_KEY"] = "3"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://vse-hackathon-foundry-s-resource.openai.azure.com"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-06-01"
os.environ["OPENAI_MODEL"] = "gpt-5-mini-deployment"  # název deploymentu v Azure

from openai import AzureOpenAI
client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

try:
    from pypdf import PdfReader
except Exception:
    from PyPDF2 import PdfReader  # fallback


def read_pdf_text(pdf_path, max_chars=50000):
    """Načte text z PDF. Pro jistotu ořízne na max_chars (kvůli limitu kontextu)."""
    if not pdf_path or not os.path.exists(pdf_path):
        return ""
    try:
        reader = PdfReader(pdf_path)
        parts = []
        for page in reader.pages:
            try:
                parts.append(page.extract_text() or "")
            except Exception:
                continue
        text = "\n".join(parts).strip()
        if max_chars and len(text) > max_chars:
            text = text[:max_chars]
        return text
    except Exception:
        return ""


def generate_evaluation_for_fulltext_ChatGPT(full_text, metadata=None):
    """
    Pošle modelu full-text (případně metadata) a vrátí striktní JSON.
    Očekává klíče: structured_evaluation a narrative_review.
    """

    if not full_text:
        return {}

    system_prompt = (
        "Your task is to perform a comprehensive evaluation of a research article. "
        "You will be provided with the FULL TEXT of an article and, potentially, other metadata "
        "such as author affiliations, journal information, or citation counts. "
        "Note that this metadata may not always be supplied. Your analysis must result in a single, valid JSON object.\n\n"
        "The JSON object must contain two top-level keys: `structured_evaluation` and `narrative_review`.\n"
        "---\n"
        "### PART 1: `structured_evaluation` Object\n"
        "---\n"
        "This object will contain a detailed breakdown of the article based on specific criteria. "
        "For each criterion listed below, provide an object containing its `value` and a one- or two-sentence `justification`.\n\n"
        "**Criteria Details:**\n\n"
        "1.  `incrementalScore`: Assesses if the work is an incremental extension (lower score) or a breakthrough (higher score).\n"
        "    -   Type: Integer, Range: [1, 5], Best: 5\n"
        "2.  `isSoftwareTool`: True if the publication's main goal is to announce software, a database, or a toolbox.\n"
        "    -   Type: Boolean\n"
        "3.  `isRegionallyFocused`: True if the study has a narrow geographical, historical, or thematic focus that limits its international relevance.\n"
        "    -   Type: Boolean, Best: False\n"
        "4.  `methodologicalRigorScore`: Evaluates the soundness and appropriateness of the research methods and experimental design.\n"
        "    -   Type: Integer, Range: [1, 5], Best: 5\n"
        "5.  `analyticalDataRigorScore`: Evaluates the depth, quality, and robustness of the data analysis and statistical methods.\n"
        "    -   Type: Integer, Range: [1, 5], Best: 5\n"
        "6.  `practicalApplicabilityScore`: Assesses the potential to apply the research in real-world scenarios (clinical, industrial, etc.).\n"
        "    -   Type: Integer, Range: [1, 5], Best: 5\n"
        "7.  `societalPolicyRelevanceScore`: Assesses the research's relevance to public health, policy, or other societal issues.\n"
        "    -   Type: Integer, Range: [1, 5], Best: 5\n"
        "8.  `futureResearchPotential`: Assesses the extent to which the paper opens new avenues for future studies.\n"
        "    -   Type: Integer, Range: [1, 5], Best: 5\n"
        "9.  `vseAuthorIsPrimary`: True if an author from VŠE (Prague University of Economics and Business) is a first or corresponding author.\n"
        "    -   Type: Boolean, Best: True\n"
        "10. `collaborationScope`: The number of unique countries in author affiliations, indicating international collaboration.\n"
        "    -   Type: Integer, Range: [1, ∞)\n"
        "11. `totalAuthors`: The total number of authors on the publication.\n"
        "    -   Type: Integer, Range: [1, ∞)\n"
        "12. `vseAuthorCount`: The number of authors affiliated with the Prague University of Economics and Business.\n"
        "    -   Type: Integer, Range: [0, ∞)\n"
        "13. `journalPrestige`: A score of the journal's perceived reputation within its field.\n"
        "    -   Type: Integer, Range: [0, 10], Best: 10\n"
        "14. `publisherPrestige`: A score of the publisher's perceived prestige in the academic community.\n"
        "    -   Type: Integer, Range: [0, 10], Best: 10\n"
        "15. `articleType`: The type of article.\n"
        "    -   Type: String, Options: [\"research_paper\", \"review\", \"editorial\", \"brief_report\", \"letter\", \"applications_note\", etc.]\n"
        "16. `pageLength`: The total number of pages in the article.\n"
        "    -   Type: Integer, Range: [1, ∞)\n\n"
        "---\n"
        "### PART 2: `narrative_review` Object\n"
        "---\n"
        "This object will contain a formal, evidence-based assessment of the research article, following the Czech M17+ evaluation style. "
        "It must have two fields: `evaluation_text` and `proposed_grade`.\n\n"
        "**Instructions for `evaluation_text`:**\n"
        "Generate a single, cohesive paragraph in a formal, objective, and direct tone. The review must be evidence-centric, grounding its assessment in the research's methodology, significance, and contribution to its field. Follow this structure:\n"
        "1.  Initial Summary & Context.\n"
        "2.  Analysis of Scientific Merit (originality, significance, difficulty).\n"
        "3.  Supporting Evidence & Critique (use metrics if available; discuss limits).\n"
        "4.  Concluding Synthesis leading to the final proposed grade.\n\n"
        "Key factors for critical assessment (M17+): lack of breakthrough innovation; nature of publication (reviews/brief reports, etc.); "
        "methodological/validation limits; limited scope/generalizability; low demonstrated impact; primarily engineering/implementation character; "
        "author contribution context.\n\n"
        "**Instructions for `proposed_grade`:** Provide an integer 1–5 per the M17+ scale described.\n\n"
        "### FINAL INSTRUCTION\n"
        "Generate only the single, valid JSON object described above. Do not include any text or formatting outside of the JSON structure itself."
    )

    user_payload = {
        "full_text": full_text,
        "two_year_mean_citedness": metadata.get("two_year_mean_citedness", None),
        "metadata": metadata or {}
    }


    response = client.chat.completions.create(
        model=os.environ.get("OPENAI_MODEL", "gpt-41-mini-deployment"),  # název Azure deploymentu
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": json.dumps(user_payload)},
        ],
    )


    output = response.choices[0].message.content
    try:
        data = json.loads(output)
    except json.JSONDecodeError:
        data = {}
    return data


PDF_DIR = "/content/drive/MyDrive/Colab Notebooks/pdfs"

rows_out = []
for row in special_df.itertuples(index=False):
    row_id = getattr(row, "id")
    abstract = getattr(row, "abstrakt", None)
    pdf_stub = getattr(row, "file_name", None)
    pdf_path = os.path.join(PDF_DIR, pdf_stub) if pdf_stub else None

    # metadata z datasetu
    metadata = {
        "title": getattr(row, "nazev", None),
        "issn": getattr(row, "issn", None),
        "doi": getattr(row, "doi", None),
        "two_year_mean_citedness": getattr(row, "2yr_mean_citedness", None),
        "had_pdf_flag": getattr(row, "pdf", None),
        "abstract": abstract
    }

    full_text = read_pdf_text(pdf_path)

    result = generate_evaluation_for_fulltext_ChatGPT(full_text, metadata=metadata)


    flat = {"id": row_id, "pdf_path": pdf_path}


    se = result.get("structured_evaluation", {}) or {}
    for k, v in se.items():
        if isinstance(v, dict):
            flat[f"{k}_value"] = v.get("value")
            flat[f"{k}_justification"] = v.get("justification")
        else:

            flat[f"{k}_value"] = v
            flat[f"{k}_justification"] = None


    nr = result.get("narrative_review", {}) or {}
    flat["narrative_evaluation_text"] = nr.get("evaluation_text")
    flat["narrative_proposed_grade"] = nr.get("proposed_grade")


    flat["raw_json"] = json.dumps(result, ensure_ascii=False)

    rows_out.append(flat)

# do DataFrame
results_df = pd.DataFrame(rows_out)


merged_df = special_df.merge(results_df, on="id", how="left")

# uložení
out_path = "/content/drive/MyDrive/Colab Notebooks/special_df_new_prompt.csv"
merged_df.to_csv(out_path, sep=",", index=False, encoding="utf-8-sig")

# náhled
merged_df.head(12)


,nazev,abstakt,file_name,id,issn,2yr_mean_citedness,pdf_path,incrementalScore_value,incrementalScore_justification,isSoftwareTool_value,...,journalPrestige_justification,publisherPrestige_value,publisherPrestige_justification,articleType_value,articleType_justification,pageLength_value,pageLength_justification,narrative_evaluation_text,narrative_proposed_grade,raw_json
0,"Beliefs, anxiety and change readiness for arti...",This study examines the change readiness for a...,special.pdf,1,1466-4399,7.171429,/content/drive/MyDrive/Colab Notebooks/pdfs/sp...,3,The paper extends established attitude and SCT...,False,...,The International Journal of Human Resource Ma...,7,Taylor & Francis (Informa) is a major academic...,research_paper,The work reports an original empirical study w...,18,The provided full text spans multiple numbered...,This article investigates HR managers' change ...,3,"{""structured_evaluation"": {""incrementalScore"":..."


In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 6.8 MB/s eta 0:00:00


In [ ]:
t = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/scored_papers_abstracts.csv", sep=";")
t.head(20)

,Citace,Nazev,ISSN,eISSN,Rok,IF - AIS,Q-AIS,SJR,Q-SJR,FORDx,...,evidence_quality,citation_quality,originality_of_methods_or_approach,readability,rigor,novelty,replicability,accessibility,grammar,m17_plus_score
0,"BAHNÍK, Štěpán, HOUDEK, Petr, VRBOVÁ, Lucie, H...",Variations on anchoring: Sequential anchoring ...,NaN,1930-2975,2019,2.163 - 1.303,50100 (Q1 D2),1.499,50100 (Q1 D2) - 50200 (Q1 D2),50204,...,4,2,5,6,4,4,4,5,7,3
1,"BARMAK, Honarvar Shakibaei Asli, FLUSSER, Jan,...",Filter-generating system of Zernike polynomials,0005-1098,NaN,2019,5.541 - 2.350,20200 (Q1 D1),3.984,20200 (Q1 D1),20205,...,3,2,5,4,4,5,3,4,3,3
2,"CAHLÍKOVÁ, Jana, CINGL, Lubomír, LEVELY, Ian. ...",How Stress Affects Performance and Competitive...,0025-1909,1526-5501,2020,4.883 - 4.068,50200 (Q1 D1),4.954,50200 (Q1 D1),50201,...,5,3,5,6,5,5,5,5,7,2
3,"CINGL, Lubomír. Social learning under acute st...",Social learning under acute stress,NaN,1932-6203,2018,2.776 - 0.978,10700 (Q2 ),1.100,10700 (Q1 D2) - 21100 (Q1 D1) - 30500 (Q1 D2) ...,50201,...,5,2,5,6,6,5,5,5,7,2
4,"ČERNÝ, Michal. Narrow big data in a stream: Co...",Narrow big data in a stream: Computational lim...,0020-0255,NaN,2019,5.910 - 1.142,10200 (Q1 D2),1.723,10200 (Q1 D1) - 20200 (Q1 D1) - 50200 (Q1 D1),10103,...,5,3,6,6,6,6,5,5,7,2
5,"DVOULETÝ, Ondřej, SRHOJ, Stjepan, PANTEA, Smar...",Public SME grants and firm performance in Euro...,0921-898X,1573-0913,2021,7.096 - 1.601,50200 (Q1 ),2.630,50200 (Q1 D1),50204,...,4,3,4,5,4,4,3,5,5,3
6,"FELDMANN, Manuel, LUKEŠ, Martin, UHLANER, Lorr...",Disentangling succession and entrepreneurship ...,0921-898X,1573-0913,2022,6.400 - 1.670,50200 (Q1 ),2.732,50200 (Q1 D1),50204,...,4,3,5,5,4,5,4,5,6,2
7,"FILIP, Ondřej, JANDA, Karel, KRIŠTOUFEK, Ladis...",Food Versus Fuel: An Updated and Expanded Evid...,0140-9883,NaN,2019,5.203 - 1.178,50200 (Q1 ),2.409,20700 (Q1 D1) - 50200 (Q1 D1),50206,...,5,4,4,6,5,4,5,6,6,2
8,"FÜRNKRANZ, Johannes, KLIEGR, Tomáš, PAULHEIM, ...",On cognitive preferences and the plausibility ...,0885-6125,1573-0565,2020,2.940 - 1.202,10200 (Q1 D2),0.667,10200 (Q2 ),10201,...,4,3,5,6,4,5,3,5,5,3
9,"HAHSLER, Michael, JOHNSON, Ian, KLIEGR, Tomáš,...","Associative Classification in R: arc, arulesCB...",2073-4859,NaN,2019,3.312 - 2.149,10100 (Q1 D1) - 10200 (Q1 D1),3.375,10100 (Q1 D1) - 50200 (Q1 D1),10201,...,3,3,3,6,3,3,4,5,7,4


In [ ]:
df_70_for_old_prompt = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset_70_new_prompt.csv", sep=",")
df_70_for_old_prompt.head()

,journal,journalfulltitle,issn,impact,ais,impactQbestW,impactQbestWcategory,aisQbestW,aisQbestWcategory,impactQbestF,...,journalPrestige_justification,publisherPrestige_value,publisherPrestige_justification,articleType_value,articleType_justification,pageLength_value,pageLength_justification,narrative_evaluation_text,narrative_proposed_grade,raw_json
0,ARTIFICIAL INTELLIGENCE,ARTIFICIAL INTELLIGENCE,0004-3702,4.6,1.726,Q2,"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1 (D2),"COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE",Q1,...,Artificial Intelligence (Elsevier) is a well-e...,8.0,Elsevier is a major academic publisher with br...,review,The manuscript is explicitly a review article ...,24.0,The article is a full-length review published ...,"This is a focused, well-referenced review that...",3.0,"{""structured_evaluation"": {""incrementalScore"":..."
1,ENVIRONMENT AND BEHAVIOR,ENVIRONMENT AND BEHAVIOR,0013-9165,4.6,1.710,Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),...,"Environment and Behavior is a well-regarded, e...",8.0,SAGE is a reputable academic publisher with st...,research_paper,The paper reports an original empirical study ...,25.0,The article spans approximately 25 pages in th...,"This preregistered, well-powered replication (...",2.0,"{""structured_evaluation"": {""incrementalScore"":..."
2,ENVIRONMENT AND BEHAVIOR,ENVIRONMENT AND BEHAVIOR,0013-9165,4.6,1.710,Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),"ENVIRONMENTAL STUDIES - PSYCHOLOGY, MULTIDISCI...",Q1 (D1),...,"Environment and Behavior is a well-regarded, r...",8.0,SAGE Publications is a respected academic publ...,research_paper,The manuscript reports original empirical stud...,25.0,The article appears in Environment and Behavio...,This paper presents two pre-registered laborat...,2.0,"{""structured_evaluation"": {""incrementalScore"":..."
3,MANAGEMENT SCIENCE,MANAGEMENT SCIENCE,0025-1909,4.9,4.180,Q1,OPERATIONS RESEARCH & MANAGEMENT SCIENCE - MAN...,Q1 (D1),OPERATIONS RESEARCH & MANAGEMENT SCIENCE - MAN...,Q1 (D2),...,"Management Science is a leading, highly select...",8.0,INFORMS is a well-respected professional socie...,research_paper,The manuscript reports original empirical rese...,16.0,"The article appears as 'Articles in Advance, p...",This experimental paper examines whether acute...,1.0,"{""structured_evaluation"": {""incrementalScore"":..."
4,REGIONAL STUDIES,REGIONAL STUDIES,0034-3404,4.2,1.343,Q1 (D1),ECONOMICS - GEOGRAPHY,Q1 (D2),ECONOMICS - ENVIRONMENTAL STUDIES - GEOGRAPHY ...,Q1 (D2),...,"Regional Studies is a well-established, reputa...",7.0,The journal is published via the Regional Stud...,editorial,The text is explicitly an editorial introducin...,5.0,The article spans pages 1513–1517 in the journ...,This editorial introduces a multidisciplinary ...,3.0,"{""structured_evaluation"": {""incrementalScore"":..."


In [ ]:
df_70_for_old_prompt.shape

(67, 75)

In [ ]:
s = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/special_df_new_prompt.csv", sep=",")
s.head()

,nazev,abstakt,file_name,id,issn,2yr_mean_citedness,pdf_path,incrementalScore_value,incrementalScore_justification,isSoftwareTool_value,...,journalPrestige_justification,publisherPrestige_value,publisherPrestige_justification,articleType_value,articleType_justification,pageLength_value,pageLength_justification,narrative_evaluation_text,narrative_proposed_grade,raw_json
0,"Beliefs, anxiety and change readiness for arti...",This study examines the change readiness for a...,special.pdf,1,1466-4399,7.171429,/content/drive/MyDrive/Colab Notebooks/pdfs/sp...,3,The paper extends established attitude and SCT...,False,...,The International Journal of Human Resource Ma...,7,Taylor & Francis (Informa) is a major academic...,research_paper,The work reports an original empirical study w...,18,The provided full text spans multiple numbered...,This article investigates HR managers' change ...,3,"{""structured_evaluation"": {""incrementalScore"":..."


In [ ]:
s.shape

(1, 42)

In [ ]:
import json
import pandas as pd
import os



def generate_categories_for_abstract_ChatGPT(abstract):
    response = client.chat.completions.create(
        model=os.environ.get("OPENAI_MODEL", "gpt-5-mini-deployment"),  # Azure deployment
        messages=[
            {
                "role": "system",
                "content": (
                    "Your job is to assign scores to an abstact of research paper in the following evaluation categories: "
                    "\"clarity\", \"field_contribution\", \"methodological_appropriateness\", \"evidence_quality\", "
                    "\"citation_quality\", \"originality_of_methods_or_approach\", \"readability\", \"rigor\", "
                    "\"novelty\", \"replicability\", \"accessibility\", \"grammar\".\n\n"

                    "Each category must be rated on a 7-point ordinal scale, with the following meaning:\n"
                    "\"1\" = \"Very low level – severe shortcomings, does not meet academic standards.\"\n"
                    "\"2\" = \"Low level – major weaknesses, limited quality.\"\n"
                    "\"3\" = \"Below average – partially acceptable, but with significant limitations.\"\n"
                    "\"4\" = \"Average – meets basic academic standards, nothing exceptional.\"\n"
                    "\"5\" = \"Above average – generally good, only minor shortcomings.\"\n"
                    "\"6\" = \"High level – solid and reliable, only very small weaknesses.\"\n"
                    "\"7\" = \"Exceptional – outstanding, meets the highest standards with no relevant flaws.\"\n\n"

                    "After rating all categories, you must also provide a predicted overall score of the paper "
                    "according to the Czech M17+ methodology for research assessment. "
                    "Choose one value from \"1\", \"2\", \"3\", \"4\", \"5\". \n"
                    "\"1\" = \"Špičkový (World-leading)\"\n"
                    "\"2\" = \"Vynikající (Excellent)\"\n"
                    "\"3\" = \"Mezinárodně uznávaný (Internationally recognized) / Velmi dobrý (Very good)\"\n"
                    "\"4\" = \"Průměrný (Average) / Národně uznatelný (Nationally recognized)\"\n"
                    "\"5\" = \"Podprůměrný (Below average)\"\n"

                    "Base your judgment only on the provided TARGET paper’s abstract. "
                    "Be consistent: always choose exactly one value (\"1\"–\"7\") for each category, and exactly one value (\"1\"–\"5\") for the overall \"m17_plus_score\". "
                    "Do not invent new categories or values.\n\n"

                    "Return strict JSON only, exactly like:\n"
                    "{\n"
                    "  \"clarity\": \"value\",\n"
                    "  \"field_contribution\": \"value\",\n"
                    "  \"methodological_appropriateness\": \"value\",\n"
                    "  \"evidence_quality\": \"value\",\n"
                    "  \"citation_quality\": \"value\",\n"
                    "  \"originality_of_methods_or_approach\": \"value\",\n"
                    "  \"readability\": \"value\",\n"
                    "  \"rigor\": \"value\",\n"
                    "  \"novelty\": \"value\",\n"
                    "  \"replicability\": \"value\",\n"
                    "  \"accessibility\": \"value\",\n"
                    "  \"grammar\": \"value\",\n"
                    "  \"m17_plus_score\": \"value\"\n"
                    "}"
                ),
            },
            {
                "role": "user",
                "content": f"Abstract of paper to be evaluated:\n\n{abstract}",
            },
        ],
    )
    output = response.choices[0].message.content
    try:
        data = json.loads(output)
    except json.JSONDecodeError:
        data = {}
    return data




In [ ]:

results = {"id": [], "old_prompt": []}

for row in s.itertuples(index=False):
    row_id = getattr(row, "id")
    abstract = getattr(row, "abstract", None)
    if abstract is None:
        abstract = ""

    scores = generate_categories_for_abstract_ChatGPT(str(abstract).strip())
    results["id"].append(row_id)
    results["old_prompt"].append(scores.get("m17_plus_score", None))

results_df = pd.DataFrame(results)

# merge podle 'id' a uložení
merged_df = s.merge(results_df, on="id", how="left")
merged_df.to_csv(
    "/content/drive/MyDrive/Colab Notebooks/special_df_new_old_prompt.csv",
    sep=",", index=False, encoding="utf-8-sig"
)
merged_df.head(12)

,nazev,abstakt,file_name,id,issn,2yr_mean_citedness,pdf_path,incrementalScore_value,incrementalScore_justification,isSoftwareTool_value,...,publisherPrestige_value,publisherPrestige_justification,articleType_value,articleType_justification,pageLength_value,pageLength_justification,narrative_evaluation_text,narrative_proposed_grade,raw_json,old_prompt
0,"Beliefs, anxiety and change readiness for arti...",This study examines the change readiness for a...,special.pdf,1,1466-4399,7.171429,/content/drive/MyDrive/Colab Notebooks/pdfs/sp...,3,The paper extends established attitude and SCT...,False,...,7,Taylor & Francis (Informa) is a major academic...,research_paper,The work reports an original empirical study w...,18,The provided full text spans multiple numbered...,This article investigates HR managers' change ...,3,"{""structured_evaluation"": {""incrementalScore"":...",5


In [ ]:
merged_df.shape

(1, 43)

In [ ]:
x = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/special_df_new_old_prompt.csv", sep=",")
x.head()

,nazev,abstakt,file_name,id,issn,2yr_mean_citedness,pdf_path,incrementalScore_value,incrementalScore_justification,isSoftwareTool_value,...,publisherPrestige_value,publisherPrestige_justification,articleType_value,articleType_justification,pageLength_value,pageLength_justification,narrative_evaluation_text,narrative_proposed_grade,raw_json,old_prompt
0,"Beliefs, anxiety and change readiness for arti...",This study examines the change readiness for a...,special.pdf,1,1466-4399,7.171429,/content/drive/MyDrive/Colab Notebooks/pdfs/sp...,3,The paper extends established attitude and SCT...,False,...,7,Taylor & Francis (Informa) is a major academic...,research_paper,The work reports an original empirical study w...,18,The provided full text spans multiple numbered...,This article investigates HR managers' change ...,3,"{""structured_evaluation"": {""incrementalScore"":...",5


In [ ]:
# cesta k pdf
/content/drive/My Drive/Colab Notebooks/pdfs/special.csv

In [ ]:
special_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/special_df.csv", sep=';')
special_df.head()

,nazev,abstract,file_name,id
0,"Beliefs, anxiety and change readiness for arti...",This study examines the change readiness for a...,NaN,1


In [ ]:
special_df['file_name'] = "special"

In [ ]:
special_df['issn'] = '1466-4399'

In [ ]:
import requests
def get_if_from_openalex(issn):
    url = f"https://api.openalex.org/sources?filter=issn:{issn}"
    response = requests.get(url)

    if response.status_code != 200:
        return None

    data = response.json()


    if not data["results"]:
        return None

    journal = data["results"][0]
    if_like = journal.get("summary_stats", {}).get("2yr_mean_citedness", None)

    return if_like

In [ ]:
special_df['2yr_mean_citedness'] = special_df['issn'].apply(get_if_from_openalex)

In [ ]:
special_df = special_df.rename(columns={'abstract': 'abstakt'})

In [ ]:
special_df['file_name'] = 'special.pdf'

In [ ]:
special_df.head()

,nazev,abstakt,file_name,id,issn,2yr_mean_citedness
0,"Beliefs, anxiety and change readiness for arti...",This study examines the change readiness for a...,special.pdf,1,1466-4399,7.171429


In [ ]:
special_df.to_csv(
    "/content/drive/MyDrive/Colab Notebooks/special.csv",
    sep=",", index=False, encoding="utf-8-sig"
)